In [38]:
pip install vk_api

Note: you may need to restart the kernel to use updated packages.


In [178]:
!pip install catboost

In [13]:
from tqdm import tqdm_notebook
from datetime import datetime, date, timedelta

import pandas as pd
import numpy as np

#from bot.config_reader import config

import os
import time
import math

import sqlite3

import tkinter
import PySimpleGUI as sg
import time
# import vk
import vk_api
from ok_api import OkApi

# Подключаем потоки
from threading import Thread

import catboost
from catboost import Pool
from catboost import CatBoostClassifier

import re

import warnings
warnings.filterwarnings('ignore')

In [7]:
# Функция создания таблицы
def create_table(name_BD, name_table, table):

    # создаём движок
    conn = sqlite3.connect(name_BD)
    table.to_sql(name_table, conn,  index=False)
    conn.close()
    
    
# Функция удаления таблицы из БД
def drop_table(name_BD, name_table):

    conn = sqlite3.connect(name_BD)
    conn.execute(f'DROP TABLE {name_table};')
    conn.commit() # фиксируем изменение
    conn.close()


# Функция получения таблицы
def get_table(name_BD, name_table):

    # создаём движок
    conn = sqlite3.connect(name_BD)    
    table = pd.read_sql(f' SELECT * FROM {name_table}', conn)
    conn.close()
    
    return table


# Функция получения строки по ндексу
def get_row_by_index(name_BD, name_table, index):

    connection = sqlite3.connect(name_BD)
    table = pd.read_sql(f'''SELECT * FROM {name_table} WHERE rowid = {index+1} ;''', connection) #+1 криво
    connection.close()
    return table   


# Функция добавления строки в БД
def insert_row(name_BD, name_table, row):

    conn = sqlite3.connect(name_BD)
    conn.execute(f'INSERT INTO {name_table} VALUES {row};')
    conn.commit() # фиксируем изменение
    conn.close()

    
# Функция получения строки по ндексу
def get_rows_by_inter(name_BD, name_table, start_ind, end_ind):

    conn = sqlite3.connect(name_BD)
    table = pd.read_sql(f'''
    SELECT * 
    FROM {name_table} 
    WHERE rowid >= {start_ind+1} AND rowid < {end_ind+1};''', conn) #+1 криво
    
    conn.close()
    return table   

In [8]:
# Функция получения ТОЛЬКО юзеров с группами
def get_with_gr(users, col_name):
    no_groups = users[col_name].isna()
    no_groups2 = users[col_name] == ''
    close_prof = users[col_name] == 'closed_profile'
    ban_prof = users[col_name] == 'banned_profile'
    have_groups = ~(no_groups | close_prof | ban_prof | no_groups2)
    
    return users[have_groups]

In [9]:
def get_stat(users, name, col_name):
    
    print(name)
    print('_____')
    
    no_groups = users[col_name].isna()
    no_groups2 = users[col_name] == ''
    close_prof = users[col_name] == 'closed_profile'
    ban_prof = users[col_name] == 'banned_profile'
    have_groups = ~(no_groups | close_prof | ban_prof | no_groups2)

    print(users[no_groups | no_groups2].shape, '- отсутствуют группы')
    print(users[close_prof].shape, '- закрытые профили')
    print(users[ban_prof].shape, '- забаненые профили')
    uesrs_gr = users[have_groups]
    print(uesrs_gr.shape, '- есть группы')
    print('_____')
    
    gr_dict = {}
    len_list = []

    for row in uesrs_gr.values:
        gr_dict[row[0]] = row[1].split(' ')
        len_list.append(len(gr_dict[row[0]]))


    print(len(len_list), '- кол-во юзеров с группами')

    len_list = np.array(len_list)

    print(len_list.max(), '- максимальное число групп')
    print(len_list.min(), '- максимальное число групп')
    print(len_list.mean(), '- среднее число групп')
    print(len_list.std(), '- квадратичное отклонение число групп')
    print('_____')

    user_list = []

    for i in range(0, 1000, 100):
        us_count = 0
        for key, v in gr_dict.items():        
            gr_count = len(v)
            if (gr_count > i) & (gr_count <= (i + 100)):
                us_count += 1 

        user_list.append(us_count)
        print(us_count, f'в диапазоне {i} - {i + 100}')
    sum(user_list)

In [10]:
def get_parsing_indexes(start_ind, end_ind, count_bot):
    
    count_record = end_ind - start_ind
    print(count_record)
    
    part = count_record / count_bot
    part = math.ceil(part)
    part

    return range(start_ind, start_ind + part*(count_bot+1), part)

In [119]:
get_row_by_index('test.db', 'indus', 0).values[0][2]

'vk1.a.tjc4ZTN8k8Tim4USMtzRo4AX3zq1qsuNDkhcB3Ea5GLUkKxwkOYNyUh3U-h84NpI_9NkDoWyxG40AoOmZHSp6XTZUkUrWGsiJRro077ysaw4QlGY0AK6yIN6eFRAKEZU_nEW-ny1hO-HdwSQ9sAgzW1BKIFQ8GfHJLBx1asKr2uliiElMAn93lBDTbtvoY0D'

In [182]:
def get_posts(indus_number):

    # получение токена доступа
    access_token = get_row_by_index('test.db', 'indus', indus_number).values[0][2]

    # создание сессии
    vk_session = vk_api.VkApi(token=access_token)
    vk = vk_session.get_api()

    # парсим то что нам нужно
    owner_id = 315012844
    wall_posts = vk.wall.get(owner_id = owner_id, count = 100)
    posts = wall_posts['items']

    return posts

# # извлекаем из ответа требуемую информацию
# reposts = ''
# for post in posts:
#     try:
#         repost = f"{post['copy_history'][0]['from_id']}_{post['copy_history'][0]['text']}"
#         reposts = f'{reposts} {repost}'
#     except (vk.exceptions.VkApiError) as e: 
#         print('нахуй')



# reposts = reposts.strip()

# death_flag = 0
# for pattern in ['розыгрыш', 'конкурс']:
#     if pattern in reposts:
#         # сохраняем ID на удаление
#         break

    

In [183]:
def get_gr_photo_url(indus_number):

    # получение токена доступа
    access_token = get_row_by_index('test.db', 'indus', indus_number).values[0][2]

    # создание сессии
    vk_session = vk_api.VkApi(token=access_token)
    vk = vk_session.get_api()

    # парсим то что нам нужно
    group_id = '213084854, 207940483'
    wall_posts = vk.groups.getById(group_ids=group_id, fields='name, photo_50 ,is_closed')

    return wall_posts

In [184]:
groups_data = get_gr_photo_url(0)
groups_data

ApiError: [5] User authorization failed: invalid access_token (4).

In [185]:
names_gr = [group['name'] for group in groups_data]
photos_gr = [group['photo_50'] for group in groups_data]
is_closed_gr = [group['is_closed'] for group in groups_data]

NameError: name 'groups_data' is not defined

In [92]:
is_closed_gr

[0, 0]

In [ ]:
inf_dict_out['names_gr']
inf_dict_out['photos_gr']
inf_dict_out['is_closed_gr']
inf_dict_out['urls_group']

In [ ]:
groups.getById

In [ ]:
	group_links[user_id].extend([g['id'] for g in api.groups.get(user_id=user_id, 
																	 extended=1, 
																	 offset=0, 
									

In [45]:
# Функция обновления значения в таблице по условию (в нашем случае статус индуса)
def update_value(name_BD, name_table, token, value):
    conn = sqlite3.connect(name_BD)
    conn.execute(f'''UPDATE {name_table} SET dead = {value} WHERE token = "{token}" ;''') #+1 криво  
    conn.commit()     
    conn.close()

In [51]:
for i in range(59):
    try:
        get_posts(i)
    except (vk_api.exceptions.ApiError) as e:
        if e.code == 5:
            print('Индус - ', i)
            print(e.code )
#             print(e.error['error_msg'])
            
            token = e.values['access_token']
            
            # метим
            update_value('test.db', 'indus', token, 1)

Индус -  3
5
Индус -  5
5
Индус -  7
5
Индус -  8
5
Индус -  29
5
Индус -  43
5
Индус -  47
5
Индус -  58
5


In [53]:
indus = get_table('test.db', 'indus')
indus[indus['dead'] == 1]

,login,passw,token,dead
3,919660065726,9660065726,vk1.a.1zNqRMPRdxmz6uoLUMxuFqES8OVfYBC5Bx6_ZHux...,1
5,919660199320,9660199320,vk1.a.v-1dX9KVHd0zQ1cgwpa0CzZuGUgKML6ksI_Gu6Rr...,1
7,919660860150,9660860150,vk1.a.N3g1rGtYYW-IiQc50Xm3Eg7LEyaP0dgcIbsCiHVh...,1
8,919660666201,9660666201,vk1.a.pMVxRFX2JKiH_5LiJnRlYpnqUz2WLtBLzWGojFI8...,1
29,919760101162,9760101162,vk1.a.9mSvk9fzADYslIB7YvGvWv6BVWYWJ542X_FoR9Zh...,1
43,919760521549,9760521549,vk1.a.dRn3VbLwTyl4wm9EHYzFxx4FwPeocl9fRXBP6Ysa...,1
47,919760546922,9760546922,vk1.a.yetqncMIKxG3gQGG9WYwjkVkR0KnYJzuF4sHJ9ms...,1
58,919760282930,9760282930,vk1.a.8lXud98NMg_Zp1Fv-4TzMmQnHIM7lT2mnJsYkDPT...,1


In [75]:
indus = get_table('/media/sidorov/dev/NoteProjects/Vybory/DataBase/test.db', 'indus')
indus.shape

(59, 4)

In [58]:
# Функция получения таблицы живых индусов
def get_rows_by_cond(name_BD, name_table):

    conn = sqlite3.connect(name_BD)
    table = pd.read_sql(f'''
    SELECT token 
    FROM {name_table} 
    WHERE dead = 0;''', conn) #+1 криво
    
    conn.close()
    return table   

In [73]:
alive_indus = get_rows_by_cond('test.db', 'indus')
number_indus = np.random.randint(0,len(alive_indus))
access_token = alive_indus.iloc[number_indus,0]
access_token

'vk1.a.p2nGl1jRN60vO8ccozUq_kMxr5JGamQ31atN5_xPEMLs6_y3YRHN9oK8vbnOKKEfxIfAyAty6-KdKPVmMpDlv_kDtaqIlNwuw1BJjyWDBG2k4xpyK_wGuyowtJf3Xu2TKNfCPDUjHlhcS-U9Ba_nMpZ6OEVsmMLCZvOwDKcy4UamQzA0oN_28cEEyD6EaMBm'

25

In [124]:
indus = get_table('test.db', 'indus')
indus

,login,passw,token
0,919660252571,9660252571,vk1.a.2x42AAf8WKfLYnFJEGJWNNfYIr4S2ZituREbbxS-...
1,919660633061,9660633061,vk1.a.tjc4ZTN8k8Tim4USMtzRo4AX3zq1qsuNDkhcB3Ea...
2,919660223575,9660223575,vk1.a.yC1zfgx3p_ioX9F3WYJqx-AJzZwwzML2Pq6L4JMh...
3,919660996992,9660996992,vk1.a.IcBi0M_01LKfa-IbJle8wV5x39zfbea2VBOHcvbc...
4,919660065726,9660065726,vk1.a.1zNqRMPRdxmz6uoLUMxuFqES8OVfYBC5Bx6_ZHux...
5,919660302550,9660302550,vk1.a.g_bjUWwia1PCheq8gLspof0pnVoEbEiyWP_6u9_Z...
6,919660199320,9660199320,vk1.a.v-1dX9KVHd0zQ1cgwpa0CzZuGUgKML6ksI_Gu6Rr...
7,919660982336,9660982336,vk1.a.p2nGl1jRN60vO8ccozUq_kMxr5JGamQ31atN5_xP...
8,919660860150,9660860150,vk1.a.N3g1rGtYYW-IiQc50Xm3Eg7LEyaP0dgcIbsCiHVh...
9,919660666201,9660666201,vk1.a.pMVxRFX2JKiH_5LiJnRlYpnqUz2WLtBLzWGojFI8...


In [126]:
# indus = indus[1:]
indus.reset_index(drop=True, inplace=True)
indus

,login,passw,token
0,919660633061,9660633061,vk1.a.tjc4ZTN8k8Tim4USMtzRo4AX3zq1qsuNDkhcB3Ea...
1,919660223575,9660223575,vk1.a.yC1zfgx3p_ioX9F3WYJqx-AJzZwwzML2Pq6L4JMh...
2,919660996992,9660996992,vk1.a.IcBi0M_01LKfa-IbJle8wV5x39zfbea2VBOHcvbc...
3,919660065726,9660065726,vk1.a.1zNqRMPRdxmz6uoLUMxuFqES8OVfYBC5Bx6_ZHux...
4,919660302550,9660302550,vk1.a.g_bjUWwia1PCheq8gLspof0pnVoEbEiyWP_6u9_Z...
5,919660199320,9660199320,vk1.a.v-1dX9KVHd0zQ1cgwpa0CzZuGUgKML6ksI_Gu6Rr...
6,919660982336,9660982336,vk1.a.p2nGl1jRN60vO8ccozUq_kMxr5JGamQ31atN5_xP...
7,919660860150,9660860150,vk1.a.N3g1rGtYYW-IiQc50Xm3Eg7LEyaP0dgcIbsCiHVh...
8,919660666201,9660666201,vk1.a.pMVxRFX2JKiH_5LiJnRlYpnqUz2WLtBLzWGojFI8...
9,77078070006,sJgdgGI300IiJeD,vk1.a.KCYgUATejkkDVwgS0hweThheD8ko7WfASmjZQfJd...


In [127]:
drop_table('test.db', 'indus')
create_table('test.db', 'indus', indus)
get_table('test.db', 'indus')

,login,passw,token
0,919660633061,9660633061,vk1.a.tjc4ZTN8k8Tim4USMtzRo4AX3zq1qsuNDkhcB3Ea...
1,919660223575,9660223575,vk1.a.yC1zfgx3p_ioX9F3WYJqx-AJzZwwzML2Pq6L4JMh...
2,919660996992,9660996992,vk1.a.IcBi0M_01LKfa-IbJle8wV5x39zfbea2VBOHcvbc...
3,919660065726,9660065726,vk1.a.1zNqRMPRdxmz6uoLUMxuFqES8OVfYBC5Bx6_ZHux...
4,919660302550,9660302550,vk1.a.g_bjUWwia1PCheq8gLspof0pnVoEbEiyWP_6u9_Z...
5,919660199320,9660199320,vk1.a.v-1dX9KVHd0zQ1cgwpa0CzZuGUgKML6ksI_Gu6Rr...
6,919660982336,9660982336,vk1.a.p2nGl1jRN60vO8ccozUq_kMxr5JGamQ31atN5_xP...
7,919660860150,9660860150,vk1.a.N3g1rGtYYW-IiQc50Xm3Eg7LEyaP0dgcIbsCiHVh...
8,919660666201,9660666201,vk1.a.pMVxRFX2JKiH_5LiJnRlYpnqUz2WLtBLzWGojFI8...
9,77078070006,sJgdgGI300IiJeD,vk1.a.KCYgUATejkkDVwgS0hweThheD8ko7WfASmjZQfJd...


# Получение ID юзера

In [13]:
indus = get_table('/media/sidorov/dev/NoteProjects/Vybory/DataBase/test.db', 'indus')
indus

,login,passw,token,dead
0,919660633061,9660633061,vk1.a.tjc4ZTN8k8Tim4USMtzRo4AX3zq1qsuNDkhcB3Ea...,1
1,919660223575,9660223575,vk1.a.yC1zfgx3p_ioX9F3WYJqx-AJzZwwzML2Pq6L4JMh...,1
2,919660996992,9660996992,vk1.a.IcBi0M_01LKfa-IbJle8wV5x39zfbea2VBOHcvbc...,1
3,919660065726,9660065726,vk1.a.1zNqRMPRdxmz6uoLUMxuFqES8OVfYBC5Bx6_ZHux...,1
4,919660302550,9660302550,vk1.a.g_bjUWwia1PCheq8gLspof0pnVoEbEiyWP_6u9_Z...,1
5,919660199320,9660199320,vk1.a.v-1dX9KVHd0zQ1cgwpa0CzZuGUgKML6ksI_Gu6Rr...,1
6,919660982336,9660982336,vk1.a.p2nGl1jRN60vO8ccozUq_kMxr5JGamQ31atN5_xP...,1
7,919660860150,9660860150,vk1.a.N3g1rGtYYW-IiQc50Xm3Eg7LEyaP0dgcIbsCiHVh...,1
8,919660666201,9660666201,vk1.a.pMVxRFX2JKiH_5LiJnRlYpnqUz2WLtBLzWGojFI8...,1
9,77078070006,sJgdgGI300IiJeD,vk1.a.KCYgUATejkkDVwgS0hweThheD8ko7WfASmjZQfJd...,1


In [135]:
len(indus)

59

In [123]:
alive = indus[indus['dead'] == 0]
alive[-5:]

,login,passw,token,dead
49,919760460457,9760460457,vk1.a.ruFT_c2AFPGI69_AdoAQ8c9veGK9hGyrVOKpfjxe...,0
50,919760468118,9760468118,vk1.a.7HedRce9_w73HVXs7GWblseTMEWfRLfId7KNkZ8K...,0
51,919760343428,9760343428,vk1.a.yFLShDsDnrHdc-dbiyNiS0i_bmd-7ux5zX4syNeD...,0
52,919760006553,9760006553,vk1.a.xUwzofkdYlzP2z69l-ttbk5S7kkMrDDJKLp7Vicy...,0
53,919760856838,9760856838,vk1.a.EAarRP4qQpIujmTPCtP8kweSa-t-OdWiTLbKbQX2...,0


In [124]:
dead = indus[indus['dead'] == 1]
dead[-5:]

,login,passw,token,dead
54,919760443738,9760443738,vk1.a.blaVS37kCbINOthSVFrIxWIz8-ungXXd7jqAObwi...,1
55,919760344459,9760344459,vk1.a.FMgIIp23CThU1uXBrY3aj05omcMKrxEuPMnz6P5h...,1
56,919760102434,9760102434,vk1.a.zi1rPXs47Kp7vTgnmrSCouVlk55pq04-R8MGbIyd...,1
57,919760686434,9760686434,vk1.a.YWBkETlIr_AWYr5-Ye6QJ0n0sMZYX9xxUdVXvw99...,1
58,919760282930,9760282930,vk1.a.8lXud98NMg_Zp1Fv-4TzMmQnHIM7lT2mnJsYkDPT...,1


In [136]:
len(dead)

24

# Нормализация ссылок 

In [ ]:
# Функция получения ID для ссылок VK
def add_VK_ids_in_DB(base, token_table, source_table, target_table, bot_i, indus_i, start_i, end_i):
    
    '''
    Функция принимает:
    - имя базы
    - имя таблицы токенов
    - имя исходной таблицы
    - имя целевой таблицы
    - номер бота
    - номер индуса
    - стартовый индекс парсинга
    - крайний индекс парсинга

    Функция выводит информацию:
    - интервал парсинга текущего бота
    - статус завершения парсинга и сколько спарсил бот всего
    - ошибку и номер индекса на котором она произошла  
    '''
    
    access_token = get_row_by_index(base, token_table, indus_i).values[0][0]

    vk_session = vk_api.VkApi(token=access_token)
    api = vk_session.get_api()


    # словарь user_ud : groups 
    group_links = {}
    parsing_count = 0

    # получение необходимого списка юзеров по заданному интервалу (сдедать через GUI)
    print(f'b_{bot_i} | интервал парсинга: {start_i} - {end_i}')  
    top20_source = get_rows_by_inter(base, source_table, start_i, end_i) # [0 ,10)

    # засекаем время парсинга
    t = time.time()
    
    for i, url_group in enumerate(tqdm_notebook(top20_source['Ссылка'].values, desc=f'b_{bot_i}')):
        
        if (stop_process == bot_i) or (stop_process == 'stop_all'):
            print(f'b_{bot_i} ОСТАНОВЛЕН!!!')
            break

        try:

            screen_name = url_group.split('com/')[1]

            answer = api.utils.resolveScreenName(screen_name = screen_name)
            group_id = answer['object_id']       
            
            # формируем кортеж для записи и заносим строку в БД
            row = (url_group, group_id)
            insert_row(base, target_table, row)                   
            parsing_count += 1
            
        except (vk_api.exceptions.VkApiError) as e:
            print('........................')
            print(e.code)
            print(f'произошёл обвал на {i} - {url_group}')
            
        time.sleep(1) 
        
    print(f'ПАРСИНГ ЗАВЕРШЁН | b_{bot_i} спарсил {parsing_count}')           

# Функция получения ID для ссылок OK
def add_OK_ids_in_DB(base, token_table, source_table, target_table, bot_i, indus_i, start_i, end_i):
    
    '''
    Функция принимает:
    - имя базы
    - имя таблицы токенов
    - имя исходной таблицы
    - имя целевой таблицы
    - номер бота
    - номер индуса
    - стартовый индекс парсинга
    - крайний индекс парсинга

    Функция выводит информацию:
    - интервал парсинга текущего бота
    - статус завершения парсинга и сколько спарсил бот всего
    - ошибку и номер индекса на котором она произошла  
    '''
    
    access_tokens = get_row_by_index(base, token_table, indus_i)
    access_token = access_tokens.values[0][0]
    application_key = access_tokens.values[0][1]
    application_secret_key = access_tokens.values[0][2]
    
    ok = OkApi(access_token=access_token, 
               application_key=application_key,
               application_secret_key=application_secret_key
              )

    # словарь user_ud : groups 
    parsing_count = 0

    # получение необходимого списка юзеров по заданному интервалу (сдедать через GUI)
    print(f'b_{bot_i} | интервал парсинга: {start_i} - {end_i}')  
    top20_source = get_rows_by_inter(base, source_table, start_i, end_i) # [0 ,10)

    # засекаем время парсинга
    t = time.time()
    
    for i, url_group in enumerate(tqdm_notebook(top20_source['Ссылка'].values, desc=f'b_{bot_i}')):
        
        if (stop_process == bot_i) or (stop_process == 'stop_all'):
            print(f'b_{bot_i} ОСТАНОВЛЕН!!!')
            break

        try:

            answer = ok.url.getInfo(url=url_group)
            group_id = answer.json()['objectId']
            
            # формируем кортеж для записи и заносим строку в БД
            row = (url_group, group_id)
            insert_row(base, target_table, row)                   
            parsing_count += 1
            
        except Exception as e:
            print('........................')
            print(e)
            print(f'произошёл обвал на {i} - {url_group}')
            
        time.sleep(1) 
        
    print(f'ПАРСИНГ ЗАВЕРШЁН | b_{bot_i} спарсил {parsing_count}')           

In [74]:
urls_top20 = pd.read_excel('/home/sidorov/Downloads/Telegram Desktop/ТОП_20_Неоф_Пабликов_1й_трехнедельный_срез_ДОРАБОТАНО.xlsx', 
#                            nrows=10)
                          )
urls_top20

,ФО,Регион,Название,Ссылка,Социальная сеть,Подписчики,Среднее количество просмотров,Место паблика в ТОП
0,ЮФО,Адыгея,Типичная Адыгея,https://vk.com/typical01,ВКонтакте,47403,4770.00,1
1,ЮФО,Адыгея,МАЙКОП АДЫГЕЯ,https://ok.ru/group/53030190055664,Одноклассники,16516,2972.88,2
2,ЮФО,Адыгея,chp_maikop,https://vk.com/public197975187,ВКонтакте,7405,2362.00,3
3,ЮФО,Адыгея,Zа наших,https://ok.ru/zanashikh,Одноклассники,3579,644.22,4
4,ЮФО,Адыгея,"О наболевшем| Музыка, советы",https://vk.com/public168784636,ВКонтакте,25596,551.00,5
...,...,...,...,...,...,...,...,...
1869,ЦФО,Ярославская область,ТУТАЕВ НОВОСТИ БЕРЕГА ONLINE,https://vk.com/beregaonline_tutaev,ВКонтакте,10259,1882.00,17
1870,ЦФО,Ярославская область,Типичный Ярославль,https://vk.com/typical_yaroslavl,ВКонтакте,24620,1760.00,18
1871,ЦФО,Ярославская область,Подслушано в Переславле,https://vk.com/overheard_pereslavl_zalessky,ВКонтакте,16157,1066.00,19
1872,ЦФО,Ярославская область,Подслушано в Тутаеве,https://vk.com/tutaev_overheard,ВКонтакте,12783,783.00,20


In [191]:
print('Общее число пабликов -', urls_top20.shape[0])
vk_top = urls_top20[urls_top20['Ссылка'].str.contains('vk.com')]
ok_top = urls_top20[urls_top20['Ссылка'].str.contains('ok.ru')]
print('Число пабликов VK -', vk_top.shape[0])
print('Число пабликов OK -', ok_top.shape[0])

Общее число пабликов - 1874
Число пабликов VK - 1615
Число пабликов OK - 259


In [113]:
# Таблица токенов для VK
VK_tokens = pd.DataFrame({'tokens' : ['vk1.a.qprhiUX1NEnCKxrcO44cIgYX86HBoQ9OGtiE3VJzHfVbZ5BnN8L8UwumDBARrRWYxANG2P_qbzF1SzjU4GA1SH9mhyqLn_mmPEPOP9tsX8wnob4fAD0Q1_FdVhbVgb91qR1IUet_pK09zwNkwYyQC3QpmQSB2KFpAceKuToS1l5HemhaPbB84H_Uv__4tbYPhCGQeyREo5yyYRWlX3fr1Q',
                                     'vk1.a.wEAR8MvNJVdCxV385t1sDZVdiSxx8iCzp0jgKKMWFRMbtU-7CrbD0zMreMuDsjUytAItgSyjyl2aDy_9j2yLuebicsMhcZ6MKA0QL1GrKutH3m5cuwwHVjL6uKuJdC4GWlDm11NWcahFdMF_FD6VoQ4z6TSuX_fAwE_UOoF23RHnMVCkeZpHtS5i6-OAN9tj5KbgyhtRJYkR-MLpkfQHfQ']})

# Таблица токенов для OK
OK_tokens = pd.DataFrame({'access_token' : ['tkn1QnyKQXDymIG278K7Qu2E99XokAsSqnu1I4l3auVt39mB9MHjxWC5G9QRsQHoc2Wh3',
                                     'tkn1cFnMJrLbR3EeEF4gbFRDk9DhQcKxGnTERvVoYFmvd6suVkwquCLNMPuY3lzYo2DFj0'],
                         'application_key' : ['CLPCMNKGDIHBABABA', 'CDHECQKGDIHBABABA'],
                         'application_secret_key' : ['f4b7c5ca2cfb6c21ab6d3051a5abbd7d', 'aad81d44261170d1e97405e577950976']
                         })

In [123]:
VK_target = pd.DataFrame(data=None, columns=['Ссылка', 'ID_new']) 
OK_target = pd.DataFrame(data=None, columns=['Ссылка', 'ID_new']) 

In [ ]:
try:
    drop_table('urls_BD.db', 'VK_tokens')
    drop_table('urls_BD.db', 'OK_tokens')
    drop_table('urls_BD.db', 'VK_source')
    drop_table('urls_BD.db', 'OK_source')
    drop_table('urls_BD.db', 'VK_target')
    drop_table('urls_BD.db', 'OK_target')
except Exception as e:
    print(e)

In [193]:
create_table('urls_BD.db', 'VK_tokens', VK_tokens)
create_table('urls_BD.db', 'OK_tokens', OK_tokens)
create_table('urls_BD.db', 'VK_source', vk_top)
create_table('urls_BD.db', 'OK_source', ok_top)
create_table('urls_BD.db', 'VK_target', VK_target)
create_table('urls_BD.db', 'OK_target', OK_target)

In [192]:
drop_table('urls_BD.db', 'VK_tokens')
drop_table('urls_BD.db', 'OK_tokens')
drop_table('urls_BD.db', 'VK_source')
drop_table('urls_BD.db', 'OK_source')
drop_table('urls_BD.db', 'VK_target')
drop_table('urls_BD.db', 'OK_target')

In [196]:
get_table('urls_BD.db', 'VK_target')

,Ссылка,ID_new


In [200]:
# флаг остановки потоков
'''
Если передан индекс - остановится поток с данным индексом
Если передано "stop_all" - остановятя ВСЕ потоки
'''
stop_process = None


In [189]:
# Нужно выполнить данную строчку, чтобы остановить все потоки!!!
# stop_process = 'stop_all'

In [201]:
# .................Инициализируем потоки.................
# группа 0 для VK
k = 0
base = 'urls_BD.db'
token_table = 'VK_tokens'
source_table = 'VK_source'
target_table = 'VK_target'
count_of_bots = 2
parsing_indexes = get_parsing_indexes(0, 10, count_of_bots)

Threads_VK = []
Threads_VK = [Thread(target=add_VK_ids_in_DB, args=(base,  token_table, 
                                            source_table, target_table,
                                            10*k + i, 10*k + i, parsing_indexes[i], parsing_indexes[i+1]))
               for i in range(count_of_bots)
              ]

# группа 0 для OK
k = 0
base = 'urls_BD.db'
token_table = 'OK_tokens'
source_table = 'OK_source'
target_table = 'OK_target'
count_of_bots = 2
parsing_indexes = get_parsing_indexes(0, 10, count_of_bots)

Threads_OK = []
Threads_OK = [Thread(target=add_OK_ids_in_DB, args=(base,  token_table, 
                                            source_table, target_table,
                                            10*k + i, 10*k + i, parsing_indexes[i], parsing_indexes[i+1]))
               for i in range(count_of_bots)
              ]


# .........Запускаем потоки (начинаем нормализацию).........
# VK
for t in Threads_VK:
    t.start()

# Проверяем живы ли потоки
for t in Threads_VK:
    print(f'Поток {t} - {t.is_alive()}')
    
# OK
for t in Threads_OK:
    t.start()

# Проверяем живы ли потоки
for t in Threads_OK:
    print(f'Поток {t} - {t.is_alive()}')
    
    
# .................Ждём завершения всех потоков
for t in Threads_VK:
    t.join()

for t in Threads_OK:
    t.join()
    

# .................Проверяем завершение всех потоков 
for t in Threads_VK:
    print(f'Поток {t} - {t.is_alive()}')
    
# Повторно проверяем живы ли потоки
for t in Threads_OK:
    print(f'Поток {t} - {t.is_alive()}') 
    
print('Нормализация окончена!')

10
10
Поток <Thread(Thread-29, started 140674716124928)> - True
Поток <Thread(Thread-30, started 140674707732224)> - True
b_0 | интервал парсинга: 0 - 5
b_1 | интервал парсинга: 5 - 10
Поток <Thread(Thread-31, started 140674699339520)> - True
Поток <Thread(Thread-32, started 140674724517632)> - True
b_0 | интервал парсинга: 0 - 5
b_1 | интервал парсинга: 5 - 10


b_1:   0%|          | 0/5 [00:00<?, ?it/s]

b_0:   0%|          | 0/5 [00:00<?, ?it/s]

b_0:   0%|          | 0/5 [00:00<?, ?it/s]

b_1:   0%|          | 0/5 [00:00<?, ?it/s]

ПАРСИНГ ЗАВЕРШЁН | b_1 спарсил 5
ПАРСИНГ ЗАВЕРШЁН | b_1 спарсил 5
ПАРСИНГ ЗАВЕРШЁН | b_0 спарсил 5
ПАРСИНГ ЗАВЕРШЁН | b_0 спарсил 5
Поток <Thread(Thread-29, stopped 140674716124928)> - False
Поток <Thread(Thread-30, stopped 140674707732224)> - False
Поток <Thread(Thread-31, stopped 140674699339520)> - False
Поток <Thread(Thread-32, stopped 140674724517632)> - False
finish


In [205]:
VK_target = get_table('urls_BD.db', 'VK_target')
OK_target = get_table('urls_BD.db', 'OK_target')

all_target = pd.concat([VK_target, OK_target])
urls_top20_new = urls_top20.merge(all_target, on='Ссылка', how='left')

urls_top20_new.to_excel('ТОП_20_Неоф_Пабликов.xlsx')

,ФО,Регион,Название,Ссылка,Социальная сеть,Подписчики,Среднее количество просмотров,Место паблика в ТОП,ID_new
0,ЮФО,Адыгея,Типичная Адыгея,https://vk.com/typical01,ВКонтакте,47403,4770.00,1,32771925
1,ЮФО,Адыгея,МАЙКОП АДЫГЕЯ,https://ok.ru/group/53030190055664,Одноклассники,16516,2972.88,2,53030190055664
2,ЮФО,Адыгея,chp_maikop,https://vk.com/public197975187,ВКонтакте,7405,2362.00,3,197975187
3,ЮФО,Адыгея,Zа наших,https://ok.ru/zanashikh,Одноклассники,3579,644.22,4,58724450172939
4,ЮФО,Адыгея,"О наболевшем| Музыка, советы",https://vk.com/public168784636,ВКонтакте,25596,551.00,5,168784636
...,...,...,...,...,...,...,...,...,...
1869,ЦФО,Ярославская область,ТУТАЕВ НОВОСТИ БЕРЕГА ONLINE,https://vk.com/beregaonline_tutaev,ВКонтакте,10259,1882.00,17,NaN
1870,ЦФО,Ярославская область,Типичный Ярославль,https://vk.com/typical_yaroslavl,ВКонтакте,24620,1760.00,18,NaN
1871,ЦФО,Ярославская область,Подслушано в Переславле,https://vk.com/overheard_pereslavl_zalessky,ВКонтакте,16157,1066.00,19,NaN
1872,ЦФО,Ярославская область,Подслушано в Тутаеве,https://vk.com/tutaev_overheard,ВКонтакте,12783,783.00,20,NaN


In [98]:
id_list = [] 

vk_top_sres = vk_top.iloc[:10]
for i, url_group in enumerate(vk_top_sres['Ссылка']):
    try:
        # получение токена доступа
        access_token = 'vk1.a.qprhiUX1NEnCKxrcO44cIgYX86HBoQ9OGtiE3VJzHfVbZ5BnN8L8UwumDBARrRWYxANG2P_qbzF1SzjU4GA1SH9mhyqLn_mmPEPOP9tsX8wnob4fAD0Q1_FdVhbVgb91qR1IUet_pK09zwNkwYyQC3QpmQSB2KFpAceKuToS1l5HemhaPbB84H_Uv__4tbYPhCGQeyREo5yyYRWlX3fr1Q'

        # создание сессии
        vk_session = vk_api.VkApi(token=access_token)
        vk = vk_session.get_api()

        # парсим то что нам нужно
        screen_name = url_group.split('com/')[1]

        answer = vk.utils.resolveScreenName(screen_name = screen_name)
        group_id = answer['object_id']
        id_list.append(group_id)
        print(group_id)
    except (vk_api.exceptions.VkApiError) as e:
        print('........................')
        print(e.code)
        print(f'произошёл обвал на {i} - {url_group}')
        
    # Выдердиваем секундную паузу во избежание блокировки    
    time.sleep(1)
    
vk_top_sres['ID_new'] = id_list
vk_top_sres = vk_top_sres[['Ссылка', 'ID_new']]

119314580
119314580
119314580
119314580
119314580
119314580
119314580
119314580
119314580
119314580


In [1]:
ok_top_sres['Ссылка']

NameError: name 'ok_top_sres' is not defined

In [94]:
id_list = [] 

# ok_top_sres = ok_top.iloc[:5]
# for i, url_group in enumerate(['https://ok.ru/profile/535607581565']):
try:
    ok = OkApi(access_token='tkn1QnyKQXDymIG278K7Qu2E99XokAsSqnu1I4l3auVt39mB9MHjxWC5G9QRsQHoc2Wh3', 
               application_key='CLPCMNKGDIHBABABA',
               application_secret_key='f4b7c5ca2cfb6c21ab6d3051a5abbd7d'
              )

#         answer = ok.url.getInfo(url=url_group)
#         group_id = answer.json()['objectId']
#         id_list.append(group_id)

    answer = ok.users.getInfo(uids=['535607581565'], 
#                              fields=['FOLLOWERS_COUNT', 'friends_count', 'age'])
 fields=('FOLLOWERS_COUNT'))
    
#     print(group_id)
except Exception as e:
    print('........................')
    print(e)
#     print(f'произошёл обвал на {i} - {url_group}')
        
        
#     time.sleep(1)
# ok_top_sres['ID_new'] = id_list
# ok_top_sres = ok_top_sres[['Ссылка', 'ID_new']]

In [95]:
answer.json()

[{'uid': '535607581565'}]

In [93]:
answer

<Response [200]>

In [ ]:
https://ok.ru/profile/535607581565

In [104]:
vk_top_sres['ID_new'] = id_list
vk_top_sres = vk_top_sres[['Ссылка', 'ID_new']]

top_sres = pd.concat([vk_top_sres, ok_top_sres])
urls_top20_new = urls_top20.merge(top_sres, on='Ссылка', how='left')
urls_top20_new

,Ссылка,ID_new
0,https://vk.com/typical01,32771925
2,https://vk.com/public197975187,197975187
4,https://vk.com/public168784636,168784636
5,https://vk.com/maykop__01,211225015
6,https://vk.com/nana_skazala,203150576
7,https://vk.com/melomanone,158927391
8,https://vk.com/club56060028,56060028
9,https://vk.com/club2033613,2033613
10,https://vk.com/mkp01,1113595
11,https://vk.com/public178420119,178420119


In [ ]:
# Функция парсинга данных
def parsing_groups(base, target_table, source_table, bot_i, indus_i, start_i, end_i):
    
    '''
    Функция принимает:
    - номер бота
    - номер индуса
    - стартовый индекс парсинга
    - крайний индекс парсинга

    Функция выводит информацию:
    - интервал парсинга текущего бота
    - статус завершения парсинга и сколько спарсил бот всего
    - ошибку и номер индекса на котором она произошла  
    '''
    
    access_token = get_row_by_index('test.db', 'indus', indus_i).values[0][2]

    vk_session = vk_api.VkApi(token=access_token)
    api = vk_session.get_api()

    ID_table = source_table
    SVOD_table = target_table # Целевая таблица, куда сохраняются данные

    # словарь user_ud : groups 
    group_links = {}
    parsing_count = 0

    # получение необходимого списка юзеров по заданному интервалу (сдедать через GUI)
    print(f'b_{bot_i} | интервал парсинга: {start_i} - {end_i}')  
    users = get_rows_by_inter(base, ID_table, start_i, end_i) # [0 ,10)

    # засекаем время парсинга
    t = time.time()
    
    for i, user in enumerate(tqdm_notebook(users['ID'].values, desc=f'b_{bot_i}')):
        
        if (stop_process == bot_i) or (stop_process == 'stop_all'):
            print(f'b_{bot_i} ОСТАНОВЛЕН!!!')
            break
        
        user_id = user #user[17:]

        try:
            group_links[user_id] = []
            
            group_links[user_id].extend([g['id'] for g in api.groups.get(user_id=user_id, 
                                                                         extended=1, 
                                                                         offset=0, 
                                                                         count=1000)['items']])
             
            groups = [str(x) for x in group_links[user_id]]
            groups = ' '.join(groups) # преобразуем список в строку формата '23378353 56106344 36959676 164811028'      

            # формируем кортеж для записи и заносим строку в БД
            row = (user_id, groups)
            insert_row(base, SVOD_table, row)                   
            parsing_count += 1
            
        time.sleep(1) 
        
    print(f'ПАРСИНГ ЗАВЕРШЁН | b_{bot_i} спарсил {parsing_count}')

In [ ]:
# группа 0

k = 0

base = '/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db'
target_table = 'train_groups'
source_table = 'train_ID'
count_of_bots = 10
parsing_indexes = get_parsing_indexes(0, 3600, count_of_bots)

Thread_list_0 = []
Thread_list_0 = [Thread(target=parsing_groups, args=(base, target_table, 
                                            source_table, 
                                            10*k + i, 10*k + i, parsing_indexes[i], parsing_indexes[i+1]))
               for i in range(count_of_bots)
              ]

for t in Thread_list_0:
    t.start()
    
for t in Thread_list_0:
    t.is_alive()

In [213]:
Thread_list_0

[<Thread(Thread-16, stopped 140674826573568)>,
 <Thread(Thread-17, stopped 140674818180864)>]

In [28]:
pip install ok-api

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from ok_api import OkApi

1


In [ ]:
from ok_api import OkApi


def main():
    """
        Простой запрос на примере метода friends.get с
        необязательным параметром типа сортировки
        (https://apiok.ru/dev/methods/rest/friends/friends.get)
    """

    ok = OkApi(access_token='OK_ACCESS_TOKEN',
               application_key='OK_APP_PUBLIC_TOKEN',
               application_secret_key='OK_APP_PRIVATE_TOKEN')

    response = ok.friends.get(sort_type='PRESENT')
    print(response.json())


if __name__ == '__main__':
    main()

In [ ]:
{"error_code":101,"error_msg":"PARAM_API_KEY : Application not exist","error_data":null}

In [17]:
for i in range(indus.shape[0]):
    try:
        # получение токена доступа
        access_token = get_row_by_index('test.db', 'indus', i).values[0][2]

        # создание сессии
        vk_session = vk_api.VkApi(token=access_token)
        vk = vk_session.get_api()

        # парсим то что нам нужно
        screen_name = 'https://vk.com/polly3_13_33'.split('com/')[1]
        wall_posts = vk.utils.resolveScreenName(screen_name = 'polly3_13_33')
        user_id = wall_posts['object_id']
    except (vk_api.exceptions.VkApiError) as e:
        print(e.code)

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [72]:
wall_posts

{'object_id': 315012844, 'type': 'user'}

In [ ]:
def 

# Получение фото юзера

In [224]:
# получение токена доступа
access_token = get_row_by_index('test.db', 'indus', 5).values[0][2]

# создание сессии
vk_session = vk_api.VkApi(token=access_token)
vk = vk_session.get_api()
bot_i = 1
i = 1

# парсим то что нам нужно
screen_name = 'https://vk.com/aspirant_ispu'.split('com/')[1]
print()
try:
    wall_posts = vk.users.get(user_ids='mekila_322', 
                          fields='is_closed,photo_200,sex,contacts,personal,bdate', 
                          lang='ru')
except (vk_api.exceptions.VkApiError) as e:
#             print('_______________________________')
#             print(e.message)
#             print(e.code)
    if e.code == 7:
        print('Коль нет прав, то пропускаем и идём дальше')

    elif e.code == 18:
        # будем добавлять 'Профиль забанен'
        groups = 'banned_profile'
        row = (user_id, groups)
        insert_row(base, SVOD_table, row)  
        parsing_count += 1
    elif e.code == 30:
        # будем добавлять 'Профиль/подписки скрыты'
        groups = 'closed_profile'
        row = (user_id, groups)
        insert_row(base, SVOD_table, row) 
        parsing_count += 1
    elif e.code == 100:
        # выводим порядковые номер пользователя и прекращаем парсинг
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(e.code)                
        print('[100] One of the parameters specified was missing or invalid: peer_id not integer')
        parsing_count += 1
        
    elif e.code == 29:
        # выводим порядковые номер пользователя и прекращаем парсинг
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print('ЛИМИТ ПО ЗАПРОСАМ')
        print(f'ПРОИЗОШЁЛ СБОЙ | b_{bot_i} остановился на {i} индексе')
         
    else:
        # выводим порядковые номер пользователя и прекращаем парсинг
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(e.code)                
        print('НЕИЗВЕСТНЫЙ СБОЙ')
        print(f'ПРОИЗОШЁЛ СБОЙ | b_{bot_i} остановился на {i} индексе')

except (vk_api.exceptions.ApiHttpError) as e:
    if e.response.status_code == 500:
        # выводим порядковые номер пользователя и прекращаем парсинг
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(e.code)                
        print('СБОЙ СВЯЗИ С СЕРВЕРОМ')
        print(f'ПРОИЗОШЁЛ СБОЙ | b_{bot_i} остановился на {i} индексе')

except (vk_api.exceptions.ApiError) as e:
    if e.response.status_code == 5:
        # выводим порядковые номер пользователя и прекращаем парсинг
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print('[5] User authorization failed: user is blocked.')
        print(e.code)                
        print('ИНДУС СДОХ')
        print(f'ИНДУС СДОХ | b_{bot_i} остановился на {i} индексе')
         
# posts = wall_posts['items']



!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
5
НЕИЗВЕСТНЫЙ СБОЙ
ПРОИЗОШЁЛ СБОЙ | b_1 остановился на 1 индексе


In [139]:
wall_posts

[{'id': 285737815,
  'bdate': '14.3',
  'photo_200': 'https://vk.com/images/camera_200.png',
  'mobile_phone': '',
  'home_phone': '',
  'sex': 2,
  'first_name': 'Егор',
  'last_name': 'Иванов',
  'can_access_closed': True,
  'is_closed': False}]

In [135]:
wall_posts

[{'id': 16524514,
  'photo_200': 'https://sun1-24.userapi.com/s/v1/ig2/av9qCA1gpI9Nlyx32_zKluVgEjRgdqOdQr7bAFsDqRv34W_0Eio5-piV4837PKoYZVhFns2wiIEHme3oRpuKTdBi.jpg?size=200x200&quality=96&crop=4,169,800,800&ava=1',
  'personal': {'alcohol': 0,
   'inspired_by': '',
   'langs': ['Русский'],
   'langs_full': [{'id': 0, 'native_name': 'Русский'}],
   'life_main': 1,
   'people_main': 0,
   'smoking': 2},
  'sex': 2,
  'first_name': 'Павел',
  'last_name': 'Колосов',
  'can_access_closed': True,
  'is_closed': False}]

In [131]:
wall_posts

[{'id': 140858796,
  'bdate': '4.5.1998',
  'photo_200': 'https://sun1-94.userapi.com/s/v1/ig2/5KwZPcKcoxgV_vTwAK4CjoLQqI9JI6FzIgE5KgoSyuBipsfz5QhnllI-lxkIkkJxQAZB3C0lrx04F_1GAZ1rX4nh.jpg?size=200x200&quality=96&crop=2,304,1197,1197&ava=1',
  'mobile_phone': '',
  'home_phone': '',
  'personal': {'alcohol': 4,
   'inspired_by': '',
   'life_main': 0,
   'people_main': 0,
   'smoking': 4},
  'sex': 2,
  'first_name': 'Александр',
  'last_name': 'Сурков',
  'can_access_closed': True,
  'is_closed': False}]

In [127]:
wall_posts

[{'id': 161830022,
  'bdate': '14.9',
  'photo_200': 'https://sun1-84.userapi.com/s/v1/ig2/CEkxzQrR809GoQUb68NjlVOysgZQBw7pRMps98cg2hDU3ScqUid8HXuj-BsbYC8JABbADLra6vviA0lKz04jRirG.jpg?size=200x200&quality=96&crop=2,7,1206,1206&ava=1',
  'mobile_phone': '89190211641',
  'home_phone': '',
  'personal': {'alcohol': 0,
   'inspired_by': 'Жизненный пиздюль',
   'langs': ['Русский'],
   'langs_full': [{'id': 0, 'native_name': 'Русский'}],
   'life_main': 0,
   'people_main': 0,
   'political': 3,
   'smoking': 2},
  'sex': 2,
  'first_name': 'Владислав',
  'last_name': 'Сидоров',
  'can_access_closed': True,
  'is_closed': False}]

In [125]:
wall_posts

[{'id': 315012844,
  'bdate': '20.12.2006',
  'photo_200': 'https://sun1-54.userapi.com/s/v1/ig2/Mcq2WCRShKwAiH4qLG7a3rzdUaxIhUABcSXqLYG5XZ2-1hNpXY10jR8G9rqj5TKsQ993pdMCuiEyxKgDhU4d4BK_.jpg?size=200x200&quality=95&crop=79,118,1355,1355&ava=1',
  'sex': 1,
  'first_name': 'Полина',
  'last_name': 'Сидорова',
  'can_access_closed': True,
  'is_closed': False}]

In [201]:
int('55')

55

In [71]:
get_table('test.db', 'indus')

,login,passw,token
0,919660252571,9660252571,vk1.a.2x42AAf8WKfLYnFJEGJWNNfYIr4S2ZituREbbxS-...
1,919660633061,9660633061,vk1.a.tjc4ZTN8k8Tim4USMtzRo4AX3zq1qsuNDkhcB3Ea...
2,919660223575,9660223575,vk1.a.yC1zfgx3p_ioX9F3WYJqx-AJzZwwzML2Pq6L4JMh...
3,919660996992,9660996992,vk1.a.IcBi0M_01LKfa-IbJle8wV5x39zfbea2VBOHcvbc...
4,919660065726,9660065726,vk1.a.1zNqRMPRdxmz6uoLUMxuFqES8OVfYBC5Bx6_ZHux...
5,919660302550,9660302550,vk1.a.g_bjUWwia1PCheq8gLspof0pnVoEbEiyWP_6u9_Z...
6,919660199320,9660199320,vk1.a.v-1dX9KVHd0zQ1cgwpa0CzZuGUgKML6ksI_Gu6Rr...
7,919660982336,9660982336,vk1.a.p2nGl1jRN60vO8ccozUq_kMxr5JGamQ31atN5_xP...
8,919660860150,9660860150,vk1.a.N3g1rGtYYW-IiQc50Xm3Eg7LEyaP0dgcIbsCiHVh...
9,919660666201,9660666201,vk1.a.pMVxRFX2JKiH_5LiJnRlYpnqUz2WLtBLzWGojFI8...


# Функция парсинга

In [225]:
# https://oauth.vk.com/authorize?client_id=6478436&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=friends&response_type=token&v=5.9

# Функция парсинга данных
def parsing_groups(base, target_table, source_table, bot_i, indus_i, start_i, end_i):
    
    '''
    Функция принимает:
    - номер бота
    - номер индуса
    - стартовый индекс парсинга
    - крайний индекс парсинга

    Функция выводит информацию:
    - интервал парсинга текущего бота
    - статус завершения парсинга и сколько спарсил бот всего
    - ошибку и номер индекса на котором она произошла  
    '''
    
    access_token = get_row_by_index('test.db', 'indus', indus_i).values[0][2]

#     v = '5.95' # Версия должна быть ИМЕННО такая!
#     #session = vk.session(access_token=access_token)
#     api = vk.API(access_token=access_token, v=v)
#     # имена таблиц с которыми работаем

    vk_session = vk_api.VkApi(token=access_token)
    api = vk_session.get_api()

    ID_table = source_table
    SVOD_table = target_table # Целевая таблица, куда сохраняются данные

    # словарь user_ud : groups 
    group_links = {}
    parsing_count = 0

    # получение необходимого списка юзеров по заданному интервалу (сдедать через GUI)
    print(f'b_{bot_i} | интервал парсинга: {start_i} - {end_i}')  
    users = get_rows_by_inter(base, ID_table, start_i, end_i) # [0 ,10)

    # засекаем время парсинга
    t = time.time()
    
    for i, user in enumerate(tqdm_notebook(users['ID'].values, desc=f'b_{bot_i}')):
        
        if (stop_process == bot_i) or (stop_process == 'stop_all'):
            print(f'b_{bot_i} ОСТАНОВЛЕН!!!')
            break
        
        user_id = user #user[17:]
        
#         cicles = api.groups.get(user_id=user_id, extended=1, count=200)['count'] // 200
#         if (cicles == 0): cicles = 1

        try:
            group_links[user_id] = []
            
#             for i in range(3):
            group_links[user_id].extend([g['id'] for g in api.groups.get(user_id=user_id, 
                                                                         extended=1, 
                                                                         offset=0, 
                                                                         count=1000)['items']])
             
#             print(group_links[user_id])
            groups = [str(x) for x in group_links[user_id]]
            groups = ' '.join(groups) # преобразуем список в строку формата '23378353 56106344 36959676 164811028'      

            # формируем кортеж для записи и заносим строку в БД
            row = (user_id, groups)
            insert_row(base, SVOD_table, row)                   
            parsing_count += 1

        # если ничего не удалось спарсить, т.е. профиль закрытый
#         except (vk.exceptions.VkAPIError) as e:
        except (vk_api.exceptions.VkApiError) as e:
#             print('_______________________________')
#             print(e.message)
#             print(e.code)
            if e.code == 7:
                print('Коль нет прав, то пропускаем и идём дальше')
                continue
            elif e.code == 18:
                # будем добавлять 'Профиль забанен'
                groups = 'banned_profile'
                row = (user_id, groups)
                insert_row(base, SVOD_table, row)  
                parsing_count += 1
            elif e.code == 30:
                # будем добавлять 'Профиль/подписки скрыты'
                groups = 'closed_profile'
                row = (user_id, groups)
                insert_row(base, SVOD_table, row) 
                parsing_count += 1
            elif e.code == 100:
                # выводим порядковые номер пользователя и прекращаем парсинг
                print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                print(e.code)                
                print('[100] One of the parameters specified was missing or invalid: peer_id not integer')
                print(f'ПРОИЗОШЁЛ СБОЙ | b_{bot_i} на {i} индексе')
            elif e.code == 29:
                # выводим порядковые номер пользователя и прекращаем парсинг
                print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                print('ЛИМИТ ПО ЗАПРОСАМ')
                print(f'ПРОИЗОШЁЛ СБОЙ | b_{bot_i} остановился на {i} индексе')
                break
            else:
                # выводим порядковые номер пользователя и прекращаем парсинг
                print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                print(e.code)                
                print('НЕИЗВЕСТНЫЙ СБОЙ')
                print(f'ПРОИЗОШЁЛ СБОЙ | b_{bot_i} остановился на {i} индексе')
                break
        except (vk_api.exceptions.ApiHttpError) as e:
            if e.response.status_code == 500:
                # выводим порядковые номер пользователя и прекращаем парсинг
                print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                print(e.code)                
                print('СБОЙ СВЯЗИ С СЕРВЕРОМ')
                print(f'ПРОИЗОШЁЛ СБОЙ | b_{bot_i} остановился на {i} индексе')
                continue # пытаемся продолжить
        except (vk_api.exceptions.ApiError) as e:
            if e.response.status_code == 5:
                # выводим порядковые номер пользователя и прекращаем парсинг
                print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                print('[5] User authorization failed: user is blocked.')
                print(e.code)                
                print('ИНДУС СДОХ')
                print(f'ИНДУС СДОХ | b_{bot_i} остановился на {i} индексе')
                break                
     
        time.sleep(1) 
        
    print(f'ПАРСИНГ ЗАВЕРШЁН | b_{bot_i} спарсил {parsing_count}')


ПАРСИНГ ЗАВЕРШЁН | b_30 спарсил 269


In [ ]:
vk_api.exceptions.ApiError

In [6]:
# https://oauth.vk.com/authorize?client_id=6478436&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=friends&response_type=token&v=5.9

# Функция парсинга данных
def parsing_friends(target_table, source_table, bot_i, indus_i, start_i, end_i):
    
    '''
    Функция принимает:
    - номер бота
    - номер индуса
    - стартовый индекс парсинга
    - крайний индекс парсинга

    Функция выводит информацию:
    - интервал парсинга текущего бота
    - статус завершения парсинга и сколько спарсил бот всего
    - ошибку и номер индекса на котором она произошла  
    '''
    
    access_token = get_row_by_index('test.db', 'indus', indus_i).values[0][2]

    v = '5.95' # Версия должна быть ИМЕННО такая!
    #session = vk.session(access_token=access_token)
    api = vk.API(access_token=access_token, v=v)
    # имена таблиц с которыми работаем
    ID_table = source_table
    SVOD_table = target_table # Целевая таблица, куда сохраняются данные

    # словарь user_ud : groups 
    group_links = {}
    parsing_count = 0

    # получение необходимого списка юзеров по заданному интервалу (сдедать через GUI)
    print(f'b_{bot_i} | интервал парсинга: {start_i} - {end_i}')  
    users = get_rows_by_inter('test.db', ID_table, start_i, end_i) # [0 ,10)

    # засекаем время парсинга
    t = time.time()
    
    for i, user in enumerate(tqdm_notebook(users['ID'].values, desc=f'b_{bot_i}')):
        
        if (stop_process == bot_i) or (stop_process == 'stop_all'):
            print(f'b_{bot_i} ОСТАНОВЛЕН!!!')
            break
        
        user_id = user #user[17:]

        try:
            group_links[user_id] = []
            group_links[user_id].extend([g for g in api.friends.get(user_id=user_id, offset=0, count=5000)['items']])
            groups = [str(x) for x in group_links[user_id]]
            groups = ' '.join(groups) # преобразуем список в строку формата '23378353 56106344 36959676 164811028'      

            # формируем кортеж для записи и заносим строку в БД
            row = (user_id, groups)
            insert_row('test.db', SVOD_table, row)                   
            parsing_count += 1

        # если ничего не удалось спарсить, т.е. профиль закрытый
        except (vk.exceptions.VkAPIError) as e:
#             print('_______________________________')
#             print(e.message)
#             print(e.code)
            if e.code == 7:
                print('Коль нет прав, то пропускаем и идём дальше')
                continue
            elif e.code == 18:
                # будем добавлять 'Профиль забанен'
                groups = 'banned_profile'
                row = (user_id, groups)
                insert_row('test.db', SVOD_table, row)  
                parsing_count += 1
            elif e.code == 30:
                # будем добавлять 'Профиль/подписки скрыты'
                groups = 'closed_profile'
                row = (user_id, groups)
                insert_row('test.db', SVOD_table, row) 
                parsing_count += 1
            elif e.code == 29:
                # выводим порядковые номер пользователя и прекращаем парсинг
                print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                print('ЛИМИТ ПО ЗАПРОСАМ')
                print(f'ПРОИЗОШЁЛ СБОЙ | b_{bot_i} остановился на {i} индексе')
                break
            else:
                # выводим порядковые номер пользователя и прекращаем парсинг
                print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                print('НЕИЗВЕСТНЫЙ СБОЙ')
                print(f'ПРОИЗОШЁЛ СБОЙ | b_{bot_i} остановился на {i} индексе')
                break

     
        time.sleep(1) 
        
    print(f'ПАРСИНГ ЗАВЕРШЁН | b_{bot_i} спарсил {parsing_count}')


In [28]:
Узнай свой психологический возраст в приложении: https://vk.com/app8209406
Я не поверила результатам, пока не прошла тест сама: https://vk.com/app51407584
💰 Узнай сколько стоит твоя страница в приложении - http://vk.com/app8227598

In [93]:
'fvfKMKMKMcdcdc'.lower()

'fvfkmkmkmcdcdc'

In [99]:
# https://oauth.vk.com/authorize?client_id=6478436&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=friends&response_type=token&v=5.9

# Функция парсинга данных
def parsing_wall(target_table, source_table, bot_i, indus_i, start_i, end_i):
    
    global vk_api
    
    '''
    Функция принимает:
    - Адрем целевой таблицы (куда пишем данные)
    - Адрес источника (перечень ID по которым происходит парсинг)
    - номер бота
    - номер индуса
    - стартовый индекс парсинга
    - крайний индекс парсинга

    Функция выводит информацию:
    - интервал парсинга текущего бота
    - статус завершения парсинга и сколько спарсил бот всего
    - ошибку и номер индекса на котором она произошла  
    '''

    # получение токена доступа
    access_token = get_row_by_index('test.db', 'indus', indus_i).values[0][2]

    # создание сессии
    vk_session = vk_api.VkApi(token=access_token)
    vk = vk_session.get_api()  

    ID_table = source_table
    SVOD_table = target_table # Целевая таблица, куда сохраняются данные

    # словарь user_ud : groups 
    group_links = {}
    parsing_count = 0

    # получение необходимого списка юзеров по заданному интервалу (сдедать через GUI)
    print(f'b_{bot_i} | интервал парсинга: {start_i} - {end_i}')  
    users = get_rows_by_inter('test.db', ID_table, start_i, end_i) # [0 ,10)

    # засекаем время парсинга
    t = time.time()
    
    for i, user in enumerate(tqdm_notebook(users['ID'].values, desc=f'b_{bot_i}')):
        
        if (stop_process == bot_i) or (stop_process == 'stop_all'):
            print(f'b_{bot_i} ОСТАНОВЛЕН!!!')
            break
        
        user_id = user #user[17:]

        try:
            group_links[user_id] = []
            
            # парсим то что нам нужно
            wall_posts = vk.wall.get(owner_id = user_id, count = 100)
            posts = wall_posts['items']
            
             # извлекаем из ответа требуемую информацию
            max_count = 100
            count = 0
            reposts = ''
            for post in posts:
                try:
                    repost = f"{post['copy_history'][0]['from_id']}_{post['copy_history'][0]['text']}" # id
                    reposts = f'{reposts} {repost}'
                    count += 1
                    if count == max_count:
                        break
                except KeyError as e:
                    pass
            reposts = reposts.strip()
            reposts = reposts.lower()
#             print(reposts)               
                 
    
            for pattern in ['розыгрыш', 'конкурс', 'https://vk.com/app8209406',
                           'https://vk.com/app51407584',
                           'http://vk.com/app8227598',
                           'дарим подарки',
                           'получи подарок',
                           'подпишись',
                           'подписывайтесь']:
                if pattern in reposts:
                    # сохраняем ID на удаление
                    row = (user_id, user_id)
                    insert_row('test.db', SVOD_table, row)                   
                    parsing_count += 1
                    break
    
#             # формируем кортеж для записи и заносим строку в БД
#             row = (user_id, reposts)
#             insert_row('test.db', SVOD_table, row)                   
#             parsing_count += 1

        # если ничего не удалось спарсить, т.е. профиль закрытый
        except (vk_api.exceptions.VkApiError) as e:
#             print('_______________________________')
#             print(e.message)
#             print(e.code)
            if e.code == 7:
                print('Коль нет прав, то пропускаем и идём дальше')
                continue
            elif e.code == 18:
                # будем добавлять 'Профиль забанен'
                groups = 'banned_profile'
                row = (user_id, groups)
                insert_row('test.db', SVOD_table, row)  
                parsing_count += 1
            elif e.code == 30:
                # будем добавлять 'Профиль/подписки скрыты'
                groups = 'closed_profile'
                row = (user_id, groups)
                insert_row('test.db', SVOD_table, row) 
                parsing_count += 1
            elif e.code == 29:
                # выводим порядковые номер пользователя и прекращаем парсинг
                print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                print('ЛИМИТ ПО ЗАПРОСАМ')
                print(f'ПРОИЗОШЁЛ СБОЙ | b_{bot_i} остановился на {i} индексе')
                break
            else:
                # выводим порядковые номер пользователя и прекращаем парсинг
                print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                print('НЕИЗВЕСТНЫЙ СБОЙ')
                print(f'ПРОИЗОШЁЛ СБОЙ | b_{bot_i} остановился на {i} индексе')
                break

     
        time.sleep(1) 
        
    print(f'ПАРСИНГ ЗАВЕРШЁН | b_{bot_i} спарсил {parsing_count}')

# Запуск парсинга

In [61]:
# флаг остановки потоков
'''
Если передан индекс - остановится поток с данным индексом
Если передано "stop_all" - остановятя ВСЕ потоки
'''
stop_process = None

In [19]:
stop_process = 'stop_all'

b_8 ОСТАНОВЛЕН!!!
ПАРСИНГ ЗАВЕРШЁН | b_8 спарсил 22
b_6 ОСТАНОВЛЕН!!!
ПАРСИНГ ЗАВЕРШЁН | b_6 спарсил 19
b_5 ОСТАНОВЛЕН!!!
ПАРСИНГ ЗАВЕРШЁН | b_5 спарсил 23
b_7 ОСТАНОВЛЕН!!!
ПАРСИНГ ЗАВЕРШЁН | b_7 спарсил 19
b_1 ОСТАНОВЛЕН!!!
ПАРСИНГ ЗАВЕРШЁН | b_1 спарсил 23
b_3 ОСТАНОВЛЕН!!!
ПАРСИНГ ЗАВЕРШЁН | b_3 спарсил 22
b_2 ОСТАНОВЛЕН!!!
ПАРСИНГ ЗАВЕРШЁН | b_2 спарсил 21
b_9 ОСТАНОВЛЕН!!!
ПАРСИНГ ЗАВЕРШЁН | b_9 спарсил 22
b_4 ОСТАНОВЛЕН!!!
ПАРСИНГ ЗАВЕРШЁН | b_4 спарсил 16
b_0 ОСТАНОВЛЕН!!!
ПАРСИНГ ЗАВЕРШЁН | b_0 спарсил 23


In [12]:
get_table('test.db', 'SR_ID')

,ID
0,666624
1,1111519
2,2575781
3,3238284
4,3503744
...,...
1978,729931987
1979,739522791
1980,742063480
1981,743719344


In [13]:
get_table('test.db', 'SR_groups')

,ID,group_URL


In [54]:
er = pd.read_csv('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/Data/Фед_партии/Лоялисты ID.csv',
            header=None)
er.columns = ['ID']
er

,ID
0,513235930
1,635516141
2,312621844
3,3162952
4,547778820
...,...
2945,666292844
2946,657429372
2947,648736200
2948,3008270


In [56]:
kprf = pd.read_csv('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/Data/Фед_партии/КПРФ ID.csv')
kprf.columns = ['ID']
kprf

,ID
0,140235996
1,367811221
2,306860718
3,391626398
4,327355761
...,...
3321,749301120
3322,749756516
3323,750267116
3324,750267317


In [57]:
create_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db',
            'ER_ID', er)
get_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db',
            'ER_ID')

,ID
0,513235930
1,635516141
2,312621844
3,3162952
4,547778820
...,...
2945,666292844
2946,657429372
2947,648736200
2948,3008270


In [58]:
create_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db',
            'KPRF_ID', kprf)
get_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db',
            'KPRF_ID')

,ID
0,140235996
1,367811221
2,306860718
3,391626398
4,327355761
...,...
3321,749301120
3322,749756516
3323,750267116
3324,750267317


In [102]:
# drop_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db', 'ER_groups')
# df = pd.DataFrame(data=None, columns=['ID', 'groups'])
# create_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db', 'ER_groups', df)
df_er = get_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db', 'ER_groups')
df_er = get_with_gr(df_er, 'groups')

In [107]:
df_er = df_er.drop_duplicates('ID')
df_er.shape

(2297, 2)

In [108]:
# drop_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db',
#             'ER_groups_clean')
create_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db',
            'ER_groups_clean', df_er)
get_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db',
            'ER_groups_clean')

,ID,groups
0,513235930,80821398 126326014 211210115 152641130
1,710804713,147573933
2,6186431,73908713 52153959 25135025 49840023 155298489 ...
3,611554323,22822305 212833279 39128795 147083625 212056594
4,339851124,35961352 191411179 24303368 215837002 49388164...
...,...,...
2292,228606343,37000377 36164349 39743760 40056206 40143205 3...
2293,379601434,181239704 36731992 163258727 187819042 1594664...
2294,104766054,179941005 135958079 82903085 37532400 17279043...
2295,218269123,28905875 8722610 140927803 29544671 73247559 2...


In [110]:
# df = pd.DataFrame(data=None, columns=['ID', 'groups'])
# create_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db', 'KPRF_groups', df)
kprf = get_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db', 'KPRF_groups')
kprf = get_with_gr(kprf, 'groups')

In [111]:
kprf = kprf.drop_duplicates('ID')
kprf.shape

(2483, 2)

In [112]:
drop_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db',
            'KPRF_groups_clean')
create_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db',
            'KPRF_groups_clean', kprf)
get_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db',
            'KPRF_groups_clean')

,ID,groups
0,7568810,20845272 206530812 74303190 29559271 35068738 ...
1,140235996,147845620 33445697 56905360 35285306 65320054 ...
2,51695569,53911449 72718092 57424472 30179569 140861544 ...
3,168159376,26362316 158209155 111597892 100455204 1592352...
4,113466091,196094923 26545333 98018205 191501182 93144070...
...,...,...
2478,300495866,158209155 107673211 198972258 71464939 1109947...
2479,404590300,28890647 35061290 23863253 15755094 58737741 4...
2480,346980078,156818651 127809291 59875474 121865642 2099225...
2481,347111145,80197218 28420497 51254145 168002791 187177136...


In [122]:
number_indus = np.random.randint(10,40)
number_indus

29

# Заменить токен для 0го индуса

In [191]:
# группа 0

k = 0

base = '/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db'
target_table = 'train_groups'
source_table = 'train_ID'
count_of_bots = 10
parsing_indexes = get_parsing_indexes(0, 3600, count_of_bots)

Thread_list_0 = []
Thread_list_0 = [Thread(target=parsing_groups, args=(base, target_table, 
                                            source_table, 
                                            10*k + i, 10*k + i, parsing_indexes[i], parsing_indexes[i+1]))
               for i in range(count_of_bots)
              ]

for t in Thread_list_0:
    t.start()
    
for t in Thread_list_0:
    t.is_alive()

3600
b_1 | интервал парсинга: 360 - 720
b_5 | интервал парсинга: 1800 - 2160
b_0 | интервал парсинга: 0 - 360
b_4 | интервал парсинга: 1440 - 1800
b_3 | интервал парсинга: 1080 - 1440
b_6 | интервал парсинга: 2160 - 2520
b_7 | интервал парсинга: 2520 - 2880
b_9 | интервал парсинга: 3240 - 3600
b_2 | интервал парсинга: 720 - 1080
b_8 | интервал парсинга: 2880 - 3240


b_5:   0%|          | 0/360 [00:00<?, ?it/s]

b_9:   0%|          | 0/360 [00:00<?, ?it/s]

b_3:   0%|          | 0/360 [00:00<?, ?it/s]

b_1:   0%|          | 0/360 [00:00<?, ?it/s]

...

b_7:   0%|          | 0/360 [00:00<?, ?it/s]

b_0:   0%|          | 0/360 [00:00<?, ?it/s]

b_4:   0%|          | 0/360 [00:00<?, ?it/s]

Exception in thread Thread-58:
Traceback (most recent call last):
  File "/tmp/ipykernel_186934/2268398583.py", line 58, in parsing_groups
  File "/media/sidorov/dev/NoteProjects/project/lib/python3.8/site-packages/vk_api/vk_api.py", line 737, in __call__
    return self._vk.method(self._method, kwargs)
  File "/media/sidorov/dev/NoteProjects/project/lib/python3.8/site-packages/vk_api/vk_api.py", line 697, in method
    raise error
vk_api.exceptions.ApiError: [5] User authorization failed: user is blocked.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_186934/2268398583.py", line 102, in parsing_groups
AttributeError: 'ApiError' object has no attribute 'message'
Exception in thread Thread-60:
Traceback (most recent ca

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [192]:
# группа 1

k = 1

base = '/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db'
target_table = 'train_groups'
source_table = 'train_ID'
count_of_bots = 10
parsing_indexes = get_parsing_indexes(3600, 7200, count_of_bots)

Thread_list_0 = []
Thread_list_0 = [Thread(target=parsing_groups, args=(base, target_table, 
                                            source_table, 
                                            10*k + i, 10*k + i, parsing_indexes[i], parsing_indexes[i+1]))
               for i in range(count_of_bots)
              ]

for t in Thread_list_0:
    t.start()
    
for t in Thread_list_0:
    t.is_alive()

3600
b_10 | интервал парсинга: 3600 - 3960
b_14 | интервал парсинга: 5040 - 5400
b_11 | интервал парсинга: 3960 - 4320
b_13 | интервал парсинга: 4680 - 5040
b_12 | интервал парсинга: 4320 - 4680
b_15 | интервал парсинга: 5400 - 5760
b_16 | интервал парсинга: 5760 - 6120
b_18 | интервал парсинга: 6480 - 6840
b_17 | интервал парсинга: 6120 - 6480
b_19 | интервал парсинга: 6840 - 7200


b_11:   0%|          | 0/360 [00:00<?, ?it/s]

b_15:   0%|          | 0/360 [00:00<?, ?it/s]

b_16:   0%|          | 0/360 [00:00<?, ?it/s]

b_12:   0%|          | 0/360 [00:00<?, ?it/s]

...

b_14:   0%|          | 0/360 [00:00<?, ?it/s]

b_10:   0%|          | 0/360 [00:00<?, ?it/s]

b_13:   0%|          | 0/360 [00:00<?, ?it/s]

b_17:   0%|          | 0/360 [00:00<?, ?it/s]

b_19:   0%|          | 0/360 [00:00<?, ?it/s]

In [193]:
# группа 2

k = 2

base = '/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db'
target_table = 'train_groups'
source_table = 'train_ID'
count_of_bots = 10
parsing_indexes = get_parsing_indexes(7200, 10800, count_of_bots)

Thread_list_0 = []
Thread_list_0 = [Thread(target=parsing_groups, args=(base, target_table, 
                                            source_table, 
                                            10*k + i, 10*k + i, parsing_indexes[i], parsing_indexes[i+1]))
               for i in range(count_of_bots)
              ]

for t in Thread_list_0:
    t.start()
    
for t in Thread_list_0:
    t.is_alive()

3600
b_21 | интервал парсинга: 7560 - 7920
b_22 | интервал парсинга: 7920 - 8280
b_25 | интервал парсинга: 9000 - 9360
b_20 | интервал парсинга: 7200 - 7560
b_24 | интервал парсинга: 8640 - 9000
b_29 | интервал парсинга: 10440 - 10800
b_26 | интервал парсинга: 9360 - 9720
b_23 | интервал парсинга: 8280 - 8640
b_28 | интервал парсинга: 10080 - 10440
b_27 | интервал парсинга: 9720 - 10080


b_20:   0%|          | 0/360 [00:00<?, ?it/s]

b_23:   0%|          | 0/360 [00:00<?, ?it/s]

b_21:   0%|          | 0/360 [00:00<?, ?it/s]

b_22:   0%|          | 0/360 [00:00<?, ?it/s]

...

b_27:   0%|          | 0/360 [00:00<?, ?it/s]

b_24:   0%|          | 0/360 [00:00<?, ?it/s]

b_29:   0%|          | 0/360 [00:00<?, ?it/s]

b_25:   0%|          | 0/360 [00:00<?, ?it/s]

b_28:   0%|          | 0/360 [00:00<?, ?it/s]

In [194]:
# группа 3

k = 3

base = '/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db'
target_table = 'train_groups'
source_table = 'train_ID'
count_of_bots = 10
parsing_indexes = get_parsing_indexes(10800, 14400, count_of_bots)

Thread_list_0 = []
Thread_list_0 = [Thread(target=parsing_groups, args=(base, target_table, 
                                            source_table, 
                                            10*k + i, 10*k + i, parsing_indexes[i], parsing_indexes[i+1]))
               for i in range(count_of_bots)
              ]

for t in Thread_list_0:
    t.start()
    
for t in Thread_list_0:
    t.is_alive()

3600
b_31 | интервал парсинга: 11160 - 11520
b_32 | интервал парсинга: 11520 - 11880
b_30 | интервал парсинга: 10800 - 11160
b_35 | интервал парсинга: 12600 - 12960
b_33 | интервал парсинга: 11880 - 12240
b_34 | интервал парсинга: 12240 - 12600
b_36 | интервал парсинга: 12960 - 13320
b_37 | интервал парсинга: 13320 - 13680
b_38 | интервал парсинга: 13680 - 14040
b_39 | интервал парсинга: 14040 - 14400


b_32:   0%|          | 0/360 [00:00<?, ?it/s]

b_31:   0%|          | 0/360 [00:00<?, ?it/s]

b_33:   0%|          | 0/360 [00:00<?, ?it/s]

b_37:   0%|          | 0/360 [00:00<?, ?it/s]

...

b_35:   0%|          | 0/360 [00:00<?, ?it/s]

b_36:   0%|          | 0/360 [00:00<?, ?it/s]

b_30:   0%|          | 0/360 [00:00<?, ?it/s]

b_39:   0%|          | 0/360 [00:00<?, ?it/s]

b_38:   0%|          | 0/360 [00:00<?, ?it/s]

In [195]:
# группа 4

k = 4

base = '/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db'
target_table = 'train_groups'
source_table = 'train_ID'
count_of_bots = 10
parsing_indexes = get_parsing_indexes(14400, 18000, count_of_bots)

Thread_list_0 = []
Thread_list_0 = [Thread(target=parsing_groups, args=(base, target_table, 
                                            source_table, 
                                            10*k + i, 10*k + i, parsing_indexes[i], parsing_indexes[i+1]))
               for i in range(count_of_bots)
              ]

for t in Thread_list_0:
    t.start()
    
for t in Thread_list_0:
    t.is_alive()

3600
b_40 | интервал парсинга: 14400 - 14760
b_42 | интервал парсинга: 15120 - 15480
b_41 | интервал парсинга: 14760 - 15120
b_43 | интервал парсинга: 15480 - 15840
b_46 | интервал парсинга: 16560 - 16920
b_44 | интервал парсинга: 15840 - 16200
b_45 | интервал парсинга: 16200 - 16560
b_48 | интервал парсинга: 17280 - 17640
b_47 | интервал парсинга: 16920 - 17280
b_49 | интервал парсинга: 17640 - 18000


b_42:   0%|          | 0/360 [00:00<?, ?it/s]

b_41:   0%|          | 0/360 [00:00<?, ?it/s]

b_40:   0%|          | 0/360 [00:00<?, ?it/s]

b_48:   0%|          | 0/360 [00:00<?, ?it/s]

...

b_49:   0%|          | 0/360 [00:00<?, ?it/s]

b_44:   0%|          | 0/360 [00:00<?, ?it/s]

b_45:   0%|          | 0/360 [00:00<?, ?it/s]

Exception in thread Thread-81:
Traceback (most recent call last):
  File "/tmp/ipykernel_186934/2268398583.py", line 58, in parsing_groups
  File "/media/sidorov/dev/NoteProjects/project/lib/python3.8/site-packages/vk_api/vk_api.py", line 737, in __call__
    return self._vk.method(self._method, kwargs)
  File "/media/sidorov/dev/NoteProjects/project/lib/python3.8/site-packages/vk_api/vk_api.py", line 697, in method
    raise error
vk_api.exceptions.ApiError: [100] One of the parameters specified was missing or invalid: user_id not integer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_186934/2268398583.py", line 102, in parsing_groups
AttributeError: 'ApiError' object has no attribute 'message'


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [196]:
# группа 5

k = 5

base = '/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db'
target_table = 'train_groups'
source_table = 'train_ID'
count_of_bots = 9
parsing_indexes = get_parsing_indexes(18000, 21600, count_of_bots)

Thread_list_0 = []
Thread_list_0 = [Thread(target=parsing_groups, args=(base, target_table, 
                                            source_table, 
                                            10*k + i, 10*k + i, parsing_indexes[i], parsing_indexes[i+1]))
               for i in range(count_of_bots)
              ]

for t in Thread_list_0:
    t.start()
    
for t in Thread_list_0:
    t.is_alive()

3600
b_50 | интервал парсинга: 18000 - 18400
b_51 | интервал парсинга: 18400 - 18800
b_54 | интервал парсинга: 19600 - 20000
b_53 | интервал парсинга: 19200 - 19600
b_52 | интервал парсинга: 18800 - 19200
b_55 | интервал парсинга: 20000 - 20400
b_56 | интервал парсинга: 20400 - 20800
b_57 | интервал парсинга: 20800 - 21200
b_58 | интервал парсинга: 21200 - 21600


b_50:   0%|          | 0/400 [00:00<?, ?it/s]

b_54:   0%|          | 0/400 [00:00<?, ?it/s]

b_52:   0%|          | 0/400 [00:00<?, ?it/s]

b_51:   0%|          | 0/400 [00:00<?, ?it/s]

...

b_56:   0%|          | 0/400 [00:00<?, ?it/s]

b_53:   0%|          | 0/400 [00:00<?, ?it/s]

b_55:   0%|          | 0/400 [00:00<?, ?it/s]

Exception in thread Thread-100:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_186934/2268398583.py", line 69, in parsing_groups
  File "/tmp/ipykernel_186934/1774380367.py", line 43, in insert_row
sqlite3.OperationalError: database is locked
Exception in thread Thread-77:
Traceback (most recent call last):
  File "/tmp/ipykernel_186934/2268398583.py", line 58, in parsing_groups
  File "/media/sidorov/dev/NoteProjects/project/lib/python3.8/site-packages/vk_api/vk_api.py", line 737, in __call__
    return self._vk.method(self._method, kwargs)
  File "/media/sidorov/dev/NoteProjects/project/lib/python3.8/site-packages/vk_api/vk_api.py", line 697, in method
    raise error
vk_api.exceptions.ApiError: [30] This profile is private

During handling of the above exception, another exception oc

ПАРСИНГ ЗАВЕРШЁН | b_8 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_2 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_1 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_4 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_0 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_3 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_9 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_6 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_16 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_11 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_17 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_18 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_14 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_12 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_15 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_10 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_19 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_13 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_29 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_27 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_22 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_30 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_36 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_33 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_20 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_31 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_38 спарсил 360
ПАРСИНГ ЗАВЕРШЁН | b_23 спарсил 360


# Добивка

In [218]:
# группа 5 (Добивочная)

# parsing_indexes = get_parsing_indexes(113600, 142000, 10)
base = '/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db'
target_table = 'train_groups'
source_table = 'train_ID'
k = 5

b_40 = Thread(target=parsing_groups, args=(base, target_table, 
                                            source_table, 
                                            10*k + 0, 10*k + 0, 1800, 2160))

b_41 = Thread(target=parsing_groups, args=(base, target_table, 
                                            source_table, 
                                            10*k + 1, 10*k + 1, 2520, 2880))

b_42 = Thread(target=parsing_groups, args=(base, target_table, 
                                            source_table, 
                                            10*k + 2, 10*k + 2, 8640, 9000))

b_43 = Thread(target=parsing_groups, args=(base, target_table, 
                                            source_table, 
                                            10*k + 3, 10*k + 3, 10080, 10440))

b_44 = Thread(target=parsing_groups, args=(base, target_table, 
                                            source_table, 
                                            10*k + 4, 10*k + 4, 16920, 17280))

# b_45 = Thread(target=parsing_groups, args=(target_table, 
#                                             source_table, 
#                                             10*k + 5, 10*k + 5, parsing_indexes[5], parsing_indexes[6]))

# b_46 = Thread(target=parsing_groups, args=(target_table, 
#                                             source_table, 
#                                             10*k + 6, 10*k + 6, parsing_indexes[6], parsing_indexes[7]))

# b_47 = Thread(target=parsing_groups, args=(target_table, 
#                                             source_table, 
#                                             10*k + 7, 10*k + 7, parsing_indexes[7], parsing_indexes[8]))

# b_48 = Thread(target=parsing_groups, args=(target_table, 
#                                             source_table, 
#                                             10*k + 8, 10*k + 8, parsing_indexes[8], parsing_indexes[9]))

# b_49 = Thread(target=parsing_groups, args=(target_table, 
#                                             source_table, 
#                                             10*k + 9, 10*k + 9, parsing_indexes[9], parsing_indexes[10]))




b_40.start()
b_41.start()
b_42.start()
b_43.start()
b_44.start()
# b_45.start()
# b_46.start()
# b_47.start()
# b_48.start()
# b_49.start()


b_51 | интервал парсинга: 2520 - 2880
b_50 | интервал парсинга: 1800 - 2160
b_54 | интервал парсинга: 16920 - 17280
b_53 | интервал парсинга: 10080 - 10440
b_52 | интервал парсинга: 8640 - 9000


b_52:   0%|          | 0/360 [00:00<?, ?it/s]

b_50:   0%|          | 0/360 [00:00<?, ?it/s]

b_53:   0%|          | 0/360 [00:00<?, ?it/s]

b_51:   0%|          | 0/360 [00:00<?, ?it/s]

b_54:   0%|          | 0/360 [00:00<?, ?it/s]

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
100
НЕИЗВЕСТНЫЙ СБОЙ
ПРОИЗОШЁЛ СБОЙ | b_53 остановился на 90 индексе
ПАРСИНГ ЗАВЕРШЁН | b_53 спарсил 90


In [260]:
base = '/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db'
target_table = 'test_groups'
source_table = 'test_ID'
k = 3


b_45 = Thread(target=parsing_groups, args=(base, target_table, 
                                            source_table, 
                                            10*k + 0, 10*k + 0, 0, 30))
b_45.start()

b_30 | интервал парсинга: 0 - 30


b_30:   0%|          | 0/20 [00:00<?, ?it/s]

ПАРСИНГ ЗАВЕРШЁН | b_30 спарсил 20


In [261]:
df_pars = get_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db', 'test_groups')
get_with_gr(df_pars, 'groups').drop_duplicates('ID')

,ID,groups
0,91567186,206935665 216082217 216081633 73442711 2160820...
1,2974856,212348018 73662138 188122733 174813512 2035900...
2,61059189,29320599 71474813 112292509 30558759 206935665...
3,179738604,148091031 214588814 215683910 143674269 724436...
4,95426457,26362316 87292497 85577673 168744671 123916919...
5,93973048,143931891 75789087 175395070 99626804 58315650...
6,862521,65960786 24098496 29534144 31963425 33118494 1...
7,57463279,126822333 11022780 90555631 167394451 19245993...
8,54972382,90552903 112292509 49517102 109198561 99937016...
9,9741212,30403728 210913552 51682102 167516624 64797227...


In [226]:
df_pars = get_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db', 'train_groups')
get_with_gr(df_pars, 'groups').drop_duplicates('ID')

,ID,groups
3,4284763,17842936 176841276 37923275 96487840 61618124 ...
4,277,28261334 210948221 147958317 162317248 4829 40...
5,194249438,214191883 198992621 215323543 215897589 592423...
6,631413520,51254145 25176770 204738917 143345100 21094822...
7,69388977,138738473 182690543 29534144 71712881 46362087...
...,...,...
21895,2190940,89393563 11504106 26127512 70683182 103192717 ...
21896,2199410,88970543 68142091 209128480 206737446 32194500...
21897,2242866,42918086 171322543 63103429 126909603 57424472...
21898,2284545,75348244 18483909 26270763 212763105 212295052...


In [228]:
df_pars = get_with_gr(df_pars, 'groups').drop_duplicates('ID')
df_pars

,ID,groups
3,4284763,17842936 176841276 37923275 96487840 61618124 ...
4,277,28261334 210948221 147958317 162317248 4829 40...
5,194249438,214191883 198992621 215323543 215897589 592423...
6,631413520,51254145 25176770 204738917 143345100 21094822...
7,69388977,138738473 182690543 29534144 71712881 46362087...
...,...,...
21895,2190940,89393563 11504106 26127512 70683182 103192717 ...
21896,2199410,88970543 68142091 209128480 206737446 32194500...
21897,2242866,42918086 171322543 63103429 126909603 57424472...
21898,2284545,75348244 18483909 26270763 212763105 212295052...


In [229]:
drop_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db', 'train_groups')
create_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db', 'train_groups', df_pars)
get_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db', 'train_groups')

,ID,groups
0,4284763,17842936 176841276 37923275 96487840 61618124 ...
1,277,28261334 210948221 147958317 162317248 4829 40...
2,194249438,214191883 198992621 215323543 215897589 592423...
3,631413520,51254145 25176770 204738917 143345100 21094822...
4,69388977,138738473 182690543 29534144 71712881 46362087...
...,...,...
15997,2190940,89393563 11504106 26127512 70683182 103192717 ...
15998,2199410,88970543 68142091 209128480 206737446 32194500...
15999,2242866,42918086 171322543 63103429 126909603 57424472...
16000,2284545,75348244 18483909 26270763 212763105 212295052...


In [ ]:
df_pars = get_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/DB/vk_users.db', 'train_groups')
df_pars

In [201]:
get_with_gr(df_pars, 'groups')

,ID,groups
3,4284763,17842936 176841276 37923275 96487840 61618124 ...
4,277,28261334 210948221 147958317 162317248 4829 40...
5,194249438,214191883 198992621 215323543 215897589 592423...
6,631413520,51254145 25176770 204738917 143345100 21094822...
7,69388977,138738473 182690543 29534144 71712881 46362087...
...,...,...
20093,291050271,39009769 28890647 32439535 43772432 40020627 4...
20095,565419160,46408089 90281149 152992737 166491431 18785658...
20097,565700607,27895931 58170807 151016287 47118092 138863694...
20098,293339815,202120091 211876126 213593861 111928019 112292...


In [215]:
get_with_gr(df_pars, 'groups').drop_duplicates('ID')

,ID,groups
3,4284763,17842936 176841276 37923275 96487840 61618124 ...
4,277,28261334 210948221 147958317 162317248 4829 40...
5,194249438,214191883 198992621 215323543 215897589 592423...
6,631413520,51254145 25176770 204738917 143345100 21094822...
7,69388977,138738473 182690543 29534144 71712881 46362087...
...,...,...
20093,291050271,39009769 28890647 32439535 43772432 40020627 4...
20095,565419160,46408089 90281149 152992737 166491431 18785658...
20097,565700607,27895931 58170807 151016287 47118092 138863694...
20098,293339815,202120091 211876126 213593861 111928019 112292...


In [22]:
SR_groups = get_table('test.db', 'SR_groups')
print(SR_groups.shape)
SR_groups = get_with_gr(SR_groups, 'group_URL')
print(SR_groups.shape)

(1983, 2)
(1332, 2)


In [ ]:
Exception in thread Thread-78:
Traceback (most recent call last):
  File "/tmp/ipykernel_186029/745535877.py", line 58, in parsing_groups
  File "/media/sidorov/dev/NoteProjects/project/lib/python3.8/site-packages/vk_api/vk_api.py", line 737, in __call__
    return self._vk.method(self._method, kwargs)
  File "/media/sidorov/dev/NoteProjects/project/lib/python3.8/site-packages/vk_api/vk_api.py", line 676, in method
    raise error
vk_api.exceptions.ApiHttpError: Response code 500

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_186029/745535877.py", line 78, in parsing_groups
AttributeError: 'ApiHttpError' object has no attribute 'code'

In [ ]:
vk_api.exceptions.ApiHttpError: Response code 500

In [ ]:
get_table()

In [ ]:
# группа 5 (Добивочная)

parsing_indexes = get_parsing_indexes(113600, 142000, 10)
target_table = '🏳️‍🌈_groups_federal'
source_table = '🏳️‍🌈_id_federal'
k = 4

b_40 = Thread(target=parsing_groups, args=(target_table, 
                                            source_table, 
                                            10*k + 0, 10*k + 0, parsing_indexes[0], parsing_indexes[1]))

b_41 = Thread(target=parsing_groups, args=(target_table, 
                                            source_table, 
                                            10*k + 1, 10*k + 1, parsing_indexes[1], parsing_indexes[2]))

b_42 = Thread(target=parsing_groups, args=(target_table, 
                                            source_table, 
                                            10*k + 2, 10*k + 2, parsing_indexes[2], parsing_indexes[3]))

b_43 = Thread(target=parsing_groups, args=(target_table, 
                                            source_table, 
                                            10*k + 3, 10*k + 3, parsing_indexes[3], parsing_indexes[4]))

b_44 = Thread(target=parsing_groups, args=(target_table, 
                                            source_table, 
                                            10*k + 4, 10*k + 4, parsing_indexes[4], parsing_indexes[5]))

b_45 = Thread(target=parsing_groups, args=(target_table, 
                                            source_table, 
                                            10*k + 5, 10*k + 5, parsing_indexes[5], parsing_indexes[6]))

b_46 = Thread(target=parsing_groups, args=(target_table, 
                                            source_table, 
                                            10*k + 6, 10*k + 6, parsing_indexes[6], parsing_indexes[7]))

b_47 = Thread(target=parsing_groups, args=(target_table, 
                                            source_table, 
                                            10*k + 7, 10*k + 7, parsing_indexes[7], parsing_indexes[8]))

b_48 = Thread(target=parsing_groups, args=(target_table, 
                                            source_table, 
                                            10*k + 8, 10*k + 8, parsing_indexes[8], parsing_indexes[9]))

b_49 = Thread(target=parsing_groups, args=(target_table, 
                                            source_table, 
                                            10*k + 9, 10*k + 9, parsing_indexes[9], parsing_indexes[10]))




b_40.start()
b_41.start()
b_42.start()
b_43.start()
b_44.start()
b_45.start()
b_46.start()
b_47.start()
b_48.start()
b_49.start()


In [44]:
b_10 = Thread(target=parsing_wall, args=('🦄_reposts', 
                                            '🦄second_filter', 
                                            10, 20, parsing_indexes[0], parsing_indexes[1]))

b_11 = Thread(target=parsing_wall, args=('🦄_reposts', 
                                            '🦄second_filter', 
                                            11, 21, parsing_indexes[1], parsing_indexes[2]))

b_12 = Thread(target=parsing_wall, args=('🦄_reposts', 
                                            '🦄second_filter', 
                                            12, 22, parsing_indexes[2], parsing_indexes[3]))

b_13 = Thread(target=parsing_wall, args=('🦄_reposts', 
                                            '🦄second_filter', 
                                            13, 23, parsing_indexes[3], parsing_indexes[4]))

b_14 = Thread(target=parsing_wall, args=('🦄_reposts', 
                                            '🦄second_filter', 
                                            14, 24, parsing_indexes[4], parsing_indexes[5]))

b_15 = Thread(target=parsing_wall, args=('🦄_reposts', 
                                            '🦄second_filter', 
                                            15, 25, parsing_indexes[5], parsing_indexes[6]))

b_16 = Thread(target=parsing_wall, args=('🦄_reposts', 
                                            '🦄second_filter', 
                                            16, 26, parsing_indexes[6], parsing_indexes[7]))

b_17 = Thread(target=parsing_wall, args=('🦄_reposts', 
                                            '🦄second_filter', 
                                            17, 27, parsing_indexes[7], parsing_indexes[8]))

b_18 = Thread(target=parsing_wall, args=('🦄_reposts', 
                                            '🦄second_filter', 
                                            18, 28, parsing_indexes[8], parsing_indexes[9]))

b_19 = Thread(target=parsing_wall, args=('🦄_reposts', 
                                            '🦄second_filter', 
                                            19, 29, parsing_indexes[9], parsing_indexes[10]))




b_10.start()
b_11.start()
b_12.start()
b_13.start()
b_14.start()
b_15.start()
b_16.start()
b_17.start()
b_18.start()
b_19.start()


b_0 | интервал парсинга: 0 - 564
b_1 | интервал парсинга: 564 - 1128
b_4 | интервал парсинга: 2256 - 2820
b_2 | интервал парсинга: 1128 - 1692
b_6 | интервал парсинга: 3384 - 3948
b_7 | интервал парсинга: 3948 - 4512
b_9 | интервал парсинга: 5076 - 5640
b_8 | интервал парсинга: 4512 - 5076
b_5 | интервал парсинга: 2820 - 3384
b_3 | интервал парсинга: 1692 - 2256


b_7:   0%|          | 0/564 [00:00<?, ?it/s]

b_9:   0%|          | 0/557 [00:00<?, ?it/s]

b_1:   0%|          | 0/564 [00:00<?, ?it/s]

b_0:   0%|          | 0/564 [00:00<?, ?it/s]

...

b_6:   0%|          | 0/564 [00:00<?, ?it/s]

b_8:   0%|          | 0/564 [00:00<?, ?it/s]

b_5:   0%|          | 0/564 [00:00<?, ?it/s]

b_3:   0%|          | 0/564 [00:00<?, ?it/s]

ПАРСИНГ ЗАВЕРШЁН | b_9 спарсил 557
ПАРСИНГ ЗАВЕРШЁН | b_7 спарсил 564
ПАРСИНГ ЗАВЕРШЁН | b_8 спарсил 564
ПАРСИНГ ЗАВЕРШЁН | b_6 спарсил 564


In [60]:
stop_process = None

In [52]:
print(b_10.is_alive())
print(b_11.is_alive())
print(b_12.is_alive())
print(b_13.is_alive())
print(b_14.is_alive())
print(b_15.is_alive())
print(b_16.is_alive())
print(b_17.is_alive())
print(b_18.is_alive())
print(b_19.is_alive())

False
False
False
False
False
False
False
False
False
False


In [ ]:
b_0 = Thread(target=parsing_wall, args=('ldpr_reposts_clean', 'ldpr_id_clean', 0, 10, 0, 160))
b_1 = Thread(target=parsing_wall, args=('ldpr_reposts_clean', 'ldpr_id_clean', 1, 11, 160, 320))
b_2 = Thread(target=parsing_wall, args=('ldpr_reposts_clean', 'ldpr_id_clean', 2, 12, 320, 480))
b_3 = Thread(target=parsing_wall, args=('ldpr_reposts_clean', 'ldpr_id_clean', 3, 13, 480, 640))
b_4 = Thread(target=parsing_wall, args=('ldpr_reposts_clean', 'ldpr_id_clean', 4, 14, 640, 800))
b_5 = Thread(target=parsing_wall, args=('ldpr_reposts_clean', 'ldpr_id_clean', 5, 15, 800, 960))
b_6 = Thread(target=parsing_wall, args=('ldpr_reposts_clean', 'ldpr_id_clean', 6, 16, 960, 1120))
b_7 = Thread(target=parsing_wall, args=('ldpr_reposts_clean', 'ldpr_id_clean', 7, 17, 1120, 1280))
b_8 = Thread(target=parsing_wall, args=('ldpr_reposts_clean', 'ldpr_id_clean', 8, 18, 1280, 1440))
b_9 = Thread(target=parsing_wall, args=('ldpr_reposts_clean', 'ldpr_id_clean', 9, 19, 1440, 1700))



b_0.start()
b_1.start()
b_2.start()
b_3.start()
b_4.start()
b_5.start()
b_6.start()
b_7.start()
b_8.start()
b_9.start()


In [23]:
get_table('test.db', '🏳️‍🌈_groups_federal')

,ID,group_URL
0,192316946,closed_profile
1,39599185,closed_profile
2,76642301,closed_profile
3,324164660,closed_profile
4,50687017,closed_profile
...,...,...
37498,267284869,closed_profile
37499,67580633,closed_profile
37500,707258115,closed_profile
37501,138584673,closed_profile


# Дешифрование короткого URL в VK

In [31]:
pip install selenium

     |████████████████████████████████| 995 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 384 kB 96.6 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 3.2 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install pyautogui

Processing /home/sidorov/.cache/pip/wheels/54/04/f7/55704a9d374ed923785a4fdc0ef00151fa25306b3b93345532/PyAutoGUI-0.9.53-py3-none-any.whl
Processing /home/sidorov/.cache/pip/wheels/a2/1b/69/dc477653ccc19228bf53af76c623e4e82e5dc1b6f78d5c4d35/pytweening-1.0.4-py3-none-any.whl
Processing /home/sidorov/.cache/pip/wheels/46/4e/a7/90774eeed70e4e93598bf4cbb1ef00c56e6bdc0fb99cc4215d/python3_xlib-0.15-py3-none-any.whl
Processing /home/sidorov/.cache/pip/wheels/91/28/0d/f4eb606d38276836237d6dc2abf094241e6db5682a126e0389/PyMsgBox-1.0.9-py3-none-any.whl
Processing /home/sidorov/.cache/pip/wheels/b1/b0/5c/9fb5beb2d6c708479a670e92f4ab3b54523a15c9cd4a46e24e/PyScreeze-0.1.28-py3-none-any.whl
Processing /home/sidorov/.cache/pip/wheels/d1/72/fb/35f403de66285df8ecca7049bcc61bfb57aba0f76d1f08f7eb/MouseInfo-0.1.3-py3-none-any.whl
Processing /home/sidorov/.cache/pip/wheels/91/e3/04/184bda98ec7f7d5f0ba1f372bcf7b3ba86325151ae1dd690fe/PyGetWindow-0.0.9-py3-none-any.whl
Processing /home/sidorov/.cache/pip/wheels

In [26]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [18]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
# import pyautogui # НЕ работает в фоне
from bs4 import BeautifulSoup
import re

In [30]:
def convert_to_norm_url(urls, headless_mode = True):
    
    norm_urls = []
    
    # options
    options = webdriver.ChromeOptions()
    options.headless = headless_mode
#     chrome_options.add_argument('--headless')

    # Initialize Chrome WebDriver
    driver = webdriver.Chrome(executable_path='/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/chromedriver_linux64/chromedriver',
                                 options=chrome_options)

    try:
        for url in urls:
            driver.get("https://regvk.com/id/")
            user_url = driver.find_element(by='id', value="enter")
            user_url.clear()
            user_url.send_keys(url)
            time.sleep(2)
        #     pyautogui.hotkey('enter')
            user_url.send_keys(Keys.ENTER)
            time.sleep(2)

            elements = driver.find_elements(By.TAG_NAME, value="td")

            norm_url = re.findall(r'-?\d+\d*', elements[1].text)[0]

            norm_urls.append(norm_url)
    except Exception as e:
        print(e)

    finally:
        driver.close()
        driver.quit()
            
    return norm_urls

In [31]:
urls = ['https://vk.com/sania105', 'https://vk.com/polly3_13_33', 'https://vk.com/aspirant_ispu']
norm_urls = convert_to_norm_url(urls, False)
print(norm_urls)

/tmp/ipykernel_128494/3622428730.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/chromedriver_linux64/chromedriver',


['140858796', '315012844', '16524514']


In [13]:
df_fr = pd.read_csv('/media/sidorov/dev/NoteProjects/Vybory/DataBase/Users_friends.txt')
df_fr

,ID,friends
0,22645540,1841754 2039064 3511467 3935720 5441227 681179...
1,116504287,9431026 11171947 11371603 11706006 11779911 12...
2,234778420,2254525 4192622 4676485 5281393 7210424 742355...
3,276829110,1346816 1683828 4115935 4416727 6395653 839301...
4,265313228,1371132 9432392 23525736 25724896 27912682 596...
...,...,...
361470,416606134,15604668 17827324 26326008 31911598 36410059 4...
361471,4437266,236348 361424 588329 638672 679260 804304 8551...
361472,182378171,1313577 2721100 3559418 4541873 4746781 509267...
361473,211457085,93979 190265 235180 496505 1417963 1934424 200...


In [14]:
df_fr = get_with_gr(df_fr, 'friends')

In [15]:
df_fr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328697 entries, 0 to 361474
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   ID       328697 non-null  int64 
 1   friends  328697 non-null  object
dtypes: int64(1), object(1)
memory usage: 7.5+ MB


In [16]:
df_fr['ID']  = df_fr['ID'].apply(lambda x: str(x))

In [17]:
df = get_table('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/results.db', 'svod_predict')
df

,ID,ИМЯ И ФАМИЛИЯ,ПОЛ,ГОРОД,ЛЕТ,predicts_kprf,predicts_ldpr,predicts_sr,Ответ_на_рассылку
0,276829110,Антон Бичурин,М,Ижевск,25.0,0.994835,0.858056,0.236473,None
1,1939330,Александр Куприянов,М,Ижевск,61.0,0.989234,0.721095,0.406970,None
2,63767868,Александр Мазунин,М,Ижевск,0.0,0.987424,0.295195,0.378451,None
3,104246803,Денис Береснев,М,Ижевск,37.0,0.986443,0.752835,0.234329,None
4,478982257,Дмитрий Калашников,М,Ижевск,0.0,0.985351,0.608983,0.359327,None
...,...,...,...,...,...,...,...,...,...
158551,170800687,None,None,None,NaN,NaN,NaN,0.028940,None
158552,278474854,None,None,None,NaN,NaN,NaN,0.023854,None
158553,331535244,None,None,None,NaN,NaN,NaN,0.022973,None
158554,535580276,None,None,None,NaN,NaN,NaN,0.021494,None


In [18]:
df = df.dropna(subset='predicts_kprf')

In [19]:
df_fr

,ID,friends
0,22645540,1841754 2039064 3511467 3935720 5441227 681179...
1,116504287,9431026 11171947 11371603 11706006 11779911 12...
2,234778420,2254525 4192622 4676485 5281393 7210424 742355...
3,276829110,1346816 1683828 4115935 4416727 6395653 839301...
4,265313228,1371132 9432392 23525736 25724896 27912682 596...
...,...,...
361470,416606134,15604668 17827324 26326008 31911598 36410059 4...
361471,4437266,236348 361424 588329 638672 679260 804304 8551...
361472,182378171,1313577 2721100 3559418 4541873 4746781 509267...
361473,211457085,93979 190265 235180 496505 1417963 1934424 200...


In [20]:
df

,ID,ИМЯ И ФАМИЛИЯ,ПОЛ,ГОРОД,ЛЕТ,predicts_kprf,predicts_ldpr,predicts_sr,Ответ_на_рассылку
0,276829110,Антон Бичурин,М,Ижевск,25.0,0.994835,0.858056,0.236473,None
1,1939330,Александр Куприянов,М,Ижевск,61.0,0.989234,0.721095,0.406970,None
2,63767868,Александр Мазунин,М,Ижевск,0.0,0.987424,0.295195,0.378451,None
3,104246803,Денис Береснев,М,Ижевск,37.0,0.986443,0.752835,0.234329,None
4,478982257,Дмитрий Калашников,М,Ижевск,0.0,0.985351,0.608983,0.359327,None
...,...,...,...,...,...,...,...,...,...
158139,124997648,Елена Вихарева,Ж,Ижевск,34.0,0.028567,0.084406,0.027331,None
158140,352597651,Ер Удмуртия,Ж,Ижевск,20.0,0.023642,0.091154,0.038835,None
158141,473588522,Сергей Юрин,М,Алнаши,40.0,0.022492,0.152633,0.048902,None
158142,604287033,Стальной-Щит Панкратион,М,Ижевск,37.0,0.022008,0.096582,0.018415,None


In [21]:
df['ГОРОД'].value_counts()

Ижевск      94258
Воткинск    11774
Глазов      11183
Сарапул      9933
Можга        5977
            ...  
Лагуново        1
Пежвай          1
Кыпка           1
Вортча          1
Кесшур          1
Name: ГОРОД, Length: 871, dtype: int64

In [22]:
# df_er_isev = df[df['ГОРОД'] == 'Ижевск'].iloc[-100:,:]
df_er_isev = df.copy()
df_er_isev = df_er_isev.merge(df_fr, on=['ID'], how='inner')
df_er_isev['PARTY'] = 'ER'
df_er_isev = df_er_isev[['ID', 'friends', 'PARTY', 'ПОЛ', 'ЛЕТ']]
df_er_isev.columns = ['ID', 'friends', 'PARTY', 'SEX', 'AGE']
df_er_isev = df_er_isev.iloc[-100:,:]
df_er_isev

,ID,friends,PARTY,SEX,AGE
155806,20708080,2773967 3193955 3215819 3223346 3361622 358735...,ER,Ж,33.0
155807,74138848,262061 490879 521220 1235616 1280722 1932522 2...,ER,Ж,0.0
155808,209176419,3646694 5434237 6005191 6310990 6753625 918892...,ER,Ж,36.0
155809,180820284,1319176 4556834 6180888 6658507 7316168 766070...,ER,Ж,39.0
155810,35835756,377694 513501 589325 726154 1041197 1356361 16...,ER,Ж,41.0
...,...,...,...,...,...
155901,124997648,290608 325742 377503 377694 379913 388665 5816...,ER,Ж,34.0
155902,352597651,64474 104665 148987 235180 276633 308603 37769...,ER,Ж,20.0
155903,473588522,357762 506393 520920 720890 806950 828189 8749...,ER,М,40.0
155904,604287033,377694 665796 887295 1438561 1551731 1620967 1...,ER,М,37.0


In [23]:
df_er_isev.to_csv('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/ER_100.csv', index=False)

In [41]:
df_kprf_votk = df[df['ГОРОД'] == 'Воткинск'].iloc[:100,:]
df_kprf_votk = df_kprf_votk.merge(df_fr, on=['ID'], how='inner')
df_kprf_votk['PARTY'] = 'KPRF'
df_kprf_votk = df_kprf_votk[['ID', 'friends', 'PARTY', 'ПОЛ', 'ЛЕТ']]
df_kprf_votk.columns = ['ID', 'friends', 'PARTY', 'SEX', 'AGE']
df_kprf_votk = df_kprf_votk[:70]
df_kprf_votk

,ID,friends,PARTY,SEX,AGE
0,101002273,3229846 3527742 3916820 5769522 6048892 678682...,KPRF,М,32.0
1,133956813,59735 1367870 4260991 4337014 4722288 4724478 ...,KPRF,М,0.0
2,543749103,6316916 10008964 17203300 22887798 25795247 46...,KPRF,М,36.0
3,621175018,703438 1352551 2888709 3040113 3816213 4029642...,KPRF,М,0.0
4,473821296,22396048 165947471 188575599 192938410 2650465...,KPRF,Ж,27.0
...,...,...,...,...,...
65,124458358,5255333 6297028 7064203 7715595 10255017 10757...,KPRF,М,33.0
66,60058166,664709 3226567 3482391 4258447 5609154 7479025...,KPRF,М,36.0
67,251529751,1179254 1869126 2009658 2039064 2400891 243369...,KPRF,М,35.0
68,4258447,174386 218223 227254 371440 559078 687585 8140...,KPRF,М,32.0


In [46]:
df_sr = df.sort_values(by=['predicts_sr'], ascending=False)
df_sr

,ID,ИМЯ И ФАМИЛИЯ,ПОЛ,ГОРОД,ЛЕТ,predicts_kprf,predicts_ldpr,predicts_sr,Ответ_на_рассылку
11508,6518545,Ирина Зеленовская,Ж,Ижевск,40.0,0.747327,0.516359,0.995818,None
64699,267728,Виталий Козырев,М,Ижевск,34.0,0.506346,0.125181,0.990115,None
123049,240168636,Валентина Захарова,Ж,Ижевск,61.0,0.269302,0.095365,0.990025,None
3780,68546958,Евгений Лукиных,М,Ижевск,36.0,0.794546,0.918577,0.988119,None
149355,139410201,Елена Голубева,Ж,Сарапул,41.0,0.190975,0.092620,0.987042,None
...,...,...,...,...,...,...,...,...,...
158104,249086838,Ирина Мамонтова,Ж,Мушковай,37.0,0.048679,0.088554,NaN,None
158105,597434804,Анастасия Аверкиева,Ж,Шаркан,30.0,0.048419,0.086671,NaN,None
158111,163994226,Марина Анисимова,Ж,Киясово,33.0,0.046546,0.076786,NaN,None
158114,497792033,Анна Голубина,Ж,Шаркан,35.0,0.045508,0.080820,NaN,None


In [47]:
df_sr_glaz = df_sr[df_sr['ГОРОД'] == 'Глазов'].iloc[:100,:]
df_sr_glaz = df_sr_glaz.merge(df_fr, on=['ID'], how='inner')
df_sr_glaz['PARTY'] = 'SR'
df_sr_glaz = df_sr_glaz[['ID', 'friends', 'PARTY', 'ПОЛ', 'ЛЕТ']]
df_sr_glaz.columns = ['ID', 'friends', 'PARTY', 'SEX', 'AGE']
df_sr_glaz = df_sr_glaz[:70]
df_sr_glaz

,ID,friends,PARTY,SEX,AGE
0,25947743,37688 188949 263002 458293 477226 530498 58064...,SR,Ж,36.0
1,283179505,135907 358766 521220 630385 686531 798229 1330...,SR,М,45.0
2,511178136,4126920 4607590 6267065 6877385 7985196 862390...,SR,М,26.0
3,192019978,1312585 6098625 7702627 14458650 23530827 2453...,SR,М,34.0
4,249713853,5329001 6452723 9175317 31328138 32104023 8706...,SR,М,59.0
...,...,...,...,...,...
65,123555255,2642905 4555392 4752654 7316168 10067615 13214...,SR,Ж,26.0
66,264268193,34490296 50951992 68584154 77861826 94784324 1...,SR,М,22.0
67,500401213,185201 570690 796783 820629 1206228 1340552 19...,SR,Ж,29.0
68,410551347,9429981 20192798 32720741 59944406 74424090 76...,SR,М,44.0


In [48]:
df_er_isev.to_csv('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/ER_70_1.csv')
df_kprf_votk.to_csv('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/KPRF_70_1.csv')
df_sr_glaz.to_csv('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/SR_70_1.csv')

# Урезаем кол-во друзей (оставляем только тех, что есть в базе)

In [28]:
df_fr

,ID,friends
0,22645540,1841754 2039064 3511467 3935720 5441227 681179...
1,116504287,9431026 11171947 11371603 11706006 11779911 12...
2,234778420,2254525 4192622 4676485 5281393 7210424 742355...
3,276829110,1346816 1683828 4115935 4416727 6395653 839301...
4,265313228,1371132 9432392 23525736 25724896 27912682 596...
...,...,...
361470,416606134,15604668 17827324 26326008 31911598 36410059 4...
361471,4437266,236348 361424 588329 638672 679260 804304 8551...
361472,182378171,1313577 2721100 3559418 4541873 4746781 509267...
361473,211457085,93979 190265 235180 496505 1417963 1934424 200...


In [39]:
df_er = pd.read_csv('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/ER_100.csv')
df_kprf = pd.read_csv('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/KPRF_100.csv')
df_sr = pd.read_csv('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/SR_100.csv')

In [40]:
df_partys = pd.concat([df_er, df_kprf, df_sr])
df_partys['ID'] = df_partys['ID'].apply(lambda x: str(x))
df_partys

,ID,friends,PARTY,SEX,AGE
0,20708080,2773967 3193955 3215819 3223346 3361622 358735...,ER,Ж,33.0
1,74138848,262061 490879 521220 1235616 1280722 1932522 2...,ER,Ж,0.0
2,209176419,3646694 5434237 6005191 6310990 6753625 918892...,ER,Ж,36.0
3,180820284,1319176 4556834 6180888 6658507 7316168 766070...,ER,Ж,39.0
4,35835756,377694 513501 589325 726154 1041197 1356361 16...,ER,Ж,41.0
...,...,...,...,...,...
95,147825705,1351538 5179779 7192177 7238815 7947867 800918...,SR,М,33.0
96,155553669,113748 1601735 1755149 2349071 2538461 2700308...,SR,М,34.0
97,135572331,1859347 8665681 17770643 19383047 21048746 218...,SR,Ж,43.0
98,428016756,847493 1321090 1416543 2095316 2717703 2901792...,SR,Ж,62.0


array(['22645540', '116504287', '234778420', ..., '182378171',
       '211457085', '464800362'], dtype=object)

In [31]:
len(static_users)

328697

In [34]:
df_partys['ID'].unique().shape

(300,)

In [42]:
df_partys

,ID,friends,PARTY,SEX,AGE
0,20708080,2773967 3193955 3215819 3223346 3361622 358735...,ER,Ж,33.0
1,74138848,262061 490879 521220 1235616 1280722 1932522 2...,ER,Ж,0.0
2,209176419,3646694 5434237 6005191 6310990 6753625 918892...,ER,Ж,36.0
3,180820284,1319176 4556834 6180888 6658507 7316168 766070...,ER,Ж,39.0
4,35835756,377694 513501 589325 726154 1041197 1356361 16...,ER,Ж,41.0
...,...,...,...,...,...
95,147825705,1351538 5179779 7192177 7238815 7947867 800918...,SR,М,33.0
96,155553669,113748 1601735 1755149 2349071 2538461 2700308...,SR,М,34.0
97,135572331,1859347 8665681 17770643 19383047 21048746 218...,SR,Ж,43.0
98,428016756,847493 1321090 1416543 2095316 2717703 2901792...,SR,Ж,62.0


In [46]:
static_users = df_fr['ID'].unique()
static_users = pd.DataFrame({'ID' : static_users})
static_users

,ID
0,22645540
1,116504287
2,234778420
3,276829110
4,265313228
...,...
328692,416606134
328693,4437266
328694,182378171
328695,211457085


In [66]:
us_fr_list = []

for us_fr in tqdm_notebook(df_partys['friends'].values):
    us_fr = us_fr.split(' ')
    us_fr = pd.DataFrame({'ID' : us_fr})
    
    us_fr = us_fr.merge(static_users, on=['ID'], how='inner')
    us_fr.drop_duplicates('ID', inplace=True)
    us_fr = " ".join(us_fr['ID'].values)
    
    us_fr_list.append(us_fr)
    

In [73]:
us_fr_list[0]

'3215819 3758130 3759667 6336639 7459249 14242945 15464004 19708618 20760713 21524786 23586975 23886766 27529834 31748498 31950706 32676423 36221232 42694098 42918030 48200886 51335022 60792810 65728437 72445054 74018046 76462603 94754752 96380829 113822433 117858814 131909041 132098440 135914525 142740663 142844881 146244076 147751756 151367369 152029311 154606740 160511604 161444264 165444597 167936387 168361515 168547489 169575991 172078274 178094709 180536355 184599422 196807041 201908151 206898109 206946139 207688449 208799201 208951531 211901203 213215134 217746801 220496043 224913899 225230071 225275794 227772435 230146530 247380556 249548790 250513117 254961219 261890898 273813284 280362856 296475677 297764458 299254280 312674533 327608406 337952207 341215303 346334598 350440997 365723768 369103091 371293610 373275377 383602939 395632914 404658206 436149445 455509712 459169662 461614255 461916342 469178352 475469858 480414691 485293577 490167605 492387213 504468701 507217895 51

In [65]:
len(df_partys['friends'].values[0])

5222

In [72]:
df_partys['friends'] = us_fr_list
df_partys

,ID,friends,PARTY,SEX,AGE
0,20708080,3215819 3758130 3759667 6336639 7459249 142429...,ER,Ж,33.0
1,74138848,521220 1932522 3566558 8326976 9341242 1033373...,ER,Ж,0.0
2,209176419,16914693 22313384 22754330 27270012 31106105 3...,ER,Ж,36.0
3,180820284,6658507 15822759 18459140 21369731 29142555 46...,ER,Ж,39.0
4,35835756,377694 2271646 2376728 4335276 4611752 7018419...,ER,Ж,41.0
...,...,...,...,...,...
95,147825705,5179779 7947867 14498639 19169262 19712313 267...,SR,М,33.0
96,155553669,113748 3382860 4125583 6348787 7985196 9798744...,SR,М,34.0
97,135572331,73666744 124768842 124823495 126742153 1320735...,SR,Ж,43.0
98,428016756,49934675 150120000 427049165 482911914 508884892,SR,Ж,62.0


In [74]:
df_er = df_partys[df_partys['PARTY'] == 'ER']
df_er

,ID,friends,PARTY,SEX,AGE
0,20708080,3215819 3758130 3759667 6336639 7459249 142429...,ER,Ж,33.0
1,74138848,521220 1932522 3566558 8326976 9341242 1033373...,ER,Ж,0.0
2,209176419,16914693 22313384 22754330 27270012 31106105 3...,ER,Ж,36.0
3,180820284,6658507 15822759 18459140 21369731 29142555 46...,ER,Ж,39.0
4,35835756,377694 2271646 2376728 4335276 4611752 7018419...,ER,Ж,41.0
...,...,...,...,...,...
95,124997648,377694 581672 969817 1025271 1145659 1393376 2...,ER,Ж,34.0
96,352597651,148987 276633 377694 735512 843552 886649 1230...,ER,Ж,20.0
97,473588522,1077276 1606086 2197254 2237000 2646820 300704...,ER,М,40.0
98,604287033,377694 1438561 1749454 2072908 2237000 2368387...,ER,М,37.0


In [75]:
df_kprf = df_partys[df_partys['PARTY'] == 'KPRF']
df_kprf

,ID,friends,PARTY,SEX,AGE
0,276829110,4115935 18375252 21886340 28532272 32996723 35...,KPRF,М,25.0
1,1939330,6443014 8461015 8656784 14052574 14101383 1558...,KPRF,М,61.0
2,63767868,1939330 2009658 2685962 4045515 4500939 459810...,KPRF,М,0.0
3,104246803,1566996 2237000 2446111 2577218 2970917 332112...,KPRF,М,37.0
4,478982257,7138430 8461015 9566686 11371454 72095128 7509...,KPRF,М,0.0
...,...,...,...,...,...
95,400849457,17722224 131782608 133247970 135855094 2163879...,KPRF,М,41.0
96,195586205,11787486 21761212 50935129 59498637 75528844 7...,KPRF,М,40.0
97,534950860,6704412 8461015 14253809 18131254 63767868 790...,KPRF,М,69.0
98,339902553,5161767 5505478 16032329 25618777 33411543 524...,KPRF,Ж,42.0


In [76]:
df_sr = df_partys[df_partys['PARTY'] == 'SR']
df_sr

,ID,friends,PARTY,SEX,AGE
0,267728,245625 313483 372051 393227 666624 742634 8062...,SR,М,34.0
1,240168636,25241177 31397712 69613353 174810435 204594523...,SR,Ж,61.0
2,68546958,14151582 24279929 27466649 31814205 31884155 3...,SR,М,36.0
3,139410201,64481791 472493392,SR,Ж,41.0
4,435791109,53092943 61457051 67613124 154603675 193814459...,SR,Ж,63.0
...,...,...,...,...,...
95,147825705,5179779 7947867 14498639 19169262 19712313 267...,SR,М,33.0
96,155553669,113748 3382860 4125583 6348787 7985196 9798744...,SR,М,34.0
97,135572331,73666744 124768842 124823495 126742153 1320735...,SR,Ж,43.0
98,428016756,49934675 150120000 427049165 482911914 508884892,SR,Ж,62.0


In [77]:
df_er.to_csv('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/ER_100_clean.csv', index=False)
df_kprf.to_csv('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/KPRF_100_clean.csv', index=False)
df_sr.to_csv('/media/sidorov/dev/NoteProjects/Vybory/Results_Streamlit/SR_100_clean.csv', index=False)

In [7]:
indus_table = get_table('test.db', 'indus')
indus_table

,login,passw,token,dead
0,919660633061,9660633061,vk1.a.tjc4ZTN8k8Tim4USMtzRo4AX3zq1qsuNDkhcB3Ea...,0
1,919660223575,9660223575,vk1.a.yC1zfgx3p_ioX9F3WYJqx-AJzZwwzML2Pq6L4JMh...,0
2,919660996992,9660996992,vk1.a.IcBi0M_01LKfa-IbJle8wV5x39zfbea2VBOHcvbc...,0
3,919660065726,9660065726,vk1.a.1zNqRMPRdxmz6uoLUMxuFqES8OVfYBC5Bx6_ZHux...,0
4,919660302550,9660302550,vk1.a.g_bjUWwia1PCheq8gLspof0pnVoEbEiyWP_6u9_Z...,0
5,919660199320,9660199320,vk1.a.v-1dX9KVHd0zQ1cgwpa0CzZuGUgKML6ksI_Gu6Rr...,0
6,919660982336,9660982336,vk1.a.p2nGl1jRN60vO8ccozUq_kMxr5JGamQ31atN5_xP...,0
7,919660860150,9660860150,vk1.a.N3g1rGtYYW-IiQc50Xm3Eg7LEyaP0dgcIbsCiHVh...,0
8,919660666201,9660666201,vk1.a.pMVxRFX2JKiH_5LiJnRlYpnqUz2WLtBLzWGojFI8...,0
9,77078070006,sJgdgGI300IiJeD,vk1.a.KCYgUATejkkDVwgS0hweThheD8ko7WfASmjZQfJd...,0


In [80]:
indus_table.to_csv('/media/sidorov/dev/NoteProjects/Vybory/DataBase/indus_table.txt')

In [6]:
# Функция получения строки по ндексу
def get_row_by_index(name_BD, name_table, index):

    connection = sqlite3.connect(name_BD)
    table = pd.read_sql(f'''SELECT * FROM {name_table} WHERE rowid = {index+1} ;''', connection) #+1 криво
    connection.close()
    return table   

In [20]:
# Функция обновления значения в таблице по условию (в нашем случае статус индуса)
def update_value(name_BD, name_table, index, value):
    conn = sqlite3.connect(name_BD)
    conn.execute(f'''UPDATE {name_table} SET dead = {value} WHERE rowid = {index+1} ;''') #+1 криво  
    conn.commit()     
    conn.close()
        
 

In [21]:
update_value('test.db', 'indus', 0, 0)

In [22]:
update_value('test.db', 'indus', 5, 0)

In [23]:
get_table('test.db', 'indus')

,login,passw,token,dead
0,919660633061,9660633061,vk1.a.tjc4ZTN8k8Tim4USMtzRo4AX3zq1qsuNDkhcB3Ea...,0
1,919660223575,9660223575,vk1.a.yC1zfgx3p_ioX9F3WYJqx-AJzZwwzML2Pq6L4JMh...,0
2,919660996992,9660996992,vk1.a.IcBi0M_01LKfa-IbJle8wV5x39zfbea2VBOHcvbc...,0
3,919660065726,9660065726,vk1.a.1zNqRMPRdxmz6uoLUMxuFqES8OVfYBC5Bx6_ZHux...,0
4,919660302550,9660302550,vk1.a.g_bjUWwia1PCheq8gLspof0pnVoEbEiyWP_6u9_Z...,0
5,919660199320,9660199320,vk1.a.v-1dX9KVHd0zQ1cgwpa0CzZuGUgKML6ksI_Gu6Rr...,0
6,919660982336,9660982336,vk1.a.p2nGl1jRN60vO8ccozUq_kMxr5JGamQ31atN5_xP...,0
7,919660860150,9660860150,vk1.a.N3g1rGtYYW-IiQc50Xm3Eg7LEyaP0dgcIbsCiHVh...,0
8,919660666201,9660666201,vk1.a.pMVxRFX2JKiH_5LiJnRlYpnqUz2WLtBLzWGojFI8...,0
9,77078070006,sJgdgGI300IiJeD,vk1.a.KCYgUATejkkDVwgS0hweThheD8ko7WfASmjZQfJd...,0


In [ ]:
UPDATE inventory
   SET quantity = quantity - daily.amt
  FROM (SELECT sum(quantity) AS amt, itemId FROM sales GROUP BY 2) AS daily
 WHERE inventory.itemId = daily.itemId;

In [84]:
# drop_table('test.db', 'indus')
# create_table('test.db', 'indus', indus_table)
# get_table('test.db', 'indus')

,login,passw,token,dead
0,919660633061,9660633061,vk1.a.tjc4ZTN8k8Tim4USMtzRo4AX3zq1qsuNDkhcB3Ea...,0
1,919660223575,9660223575,vk1.a.yC1zfgx3p_ioX9F3WYJqx-AJzZwwzML2Pq6L4JMh...,0
2,919660996992,9660996992,vk1.a.IcBi0M_01LKfa-IbJle8wV5x39zfbea2VBOHcvbc...,0
3,919660065726,9660065726,vk1.a.1zNqRMPRdxmz6uoLUMxuFqES8OVfYBC5Bx6_ZHux...,0
4,919660302550,9660302550,vk1.a.g_bjUWwia1PCheq8gLspof0pnVoEbEiyWP_6u9_Z...,0
5,919660199320,9660199320,vk1.a.v-1dX9KVHd0zQ1cgwpa0CzZuGUgKML6ksI_Gu6Rr...,0
6,919660982336,9660982336,vk1.a.p2nGl1jRN60vO8ccozUq_kMxr5JGamQ31atN5_xP...,0
7,919660860150,9660860150,vk1.a.N3g1rGtYYW-IiQc50Xm3Eg7LEyaP0dgcIbsCiHVh...,0
8,919660666201,9660666201,vk1.a.pMVxRFX2JKiH_5LiJnRlYpnqUz2WLtBLzWGojFI8...,0
9,77078070006,sJgdgGI300IiJeD,vk1.a.KCYgUATejkkDVwgS0hweThheD8ko7WfASmjZQfJd...,0


In [ ]:
/media/sidorov/dev/NoteProjects/Vybory/DataBase/Users_friends.txt

In [93]:
ind = get_table('/media/sidorov/dev/NoteProjects/Vybory/DataBase/test.db', 'indus')
ind

alive_indus = ind[ind['dead'] == 0]
alive_indus

dead = ind[ind['dead'] == 1]
dead

parser_koef = len(dead)/len(ind)
parser_koef = np.round(parser_koef, 2)
parser_koef = int(parser_koef * 100)
parser_koef = f'{parser_koef}%'
parser_koef

,login,passw,token,dead
0,919660633061,9660633061,vk1.a.tjc4ZTN8k8Tim4USMtzRo4AX3zq1qsuNDkhcB3Ea...,1
1,919660223575,9660223575,vk1.a.yC1zfgx3p_ioX9F3WYJqx-AJzZwwzML2Pq6L4JMh...,1
2,919660996992,9660996992,vk1.a.IcBi0M_01LKfa-IbJle8wV5x39zfbea2VBOHcvbc...,0
3,919660065726,9660065726,vk1.a.1zNqRMPRdxmz6uoLUMxuFqES8OVfYBC5Bx6_ZHux...,1
4,919660302550,9660302550,vk1.a.g_bjUWwia1PCheq8gLspof0pnVoEbEiyWP_6u9_Z...,1
5,919660199320,9660199320,vk1.a.v-1dX9KVHd0zQ1cgwpa0CzZuGUgKML6ksI_Gu6Rr...,1
6,919660982336,9660982336,vk1.a.p2nGl1jRN60vO8ccozUq_kMxr5JGamQ31atN5_xP...,1
7,919660860150,9660860150,vk1.a.N3g1rGtYYW-IiQc50Xm3Eg7LEyaP0dgcIbsCiHVh...,1
8,919660666201,9660666201,vk1.a.pMVxRFX2JKiH_5LiJnRlYpnqUz2WLtBLzWGojFI8...,1
9,77078070006,sJgdgGI300IiJeD,vk1.a.KCYgUATejkkDVwgS0hweThheD8ko7WfASmjZQfJd...,0


In [106]:
while True:
    try:
        x = int(input("Please enter a number: "))
    except ValueError:
        print("Oops!  That was not a valid number.  Try again...")
    else:
        print('Thanks,',x,'is indeed an integer')
        break

Please enter a number: f
Oops!  That was not a valid number.  Try again...
Please enter a number: f
Oops!  That was not a valid number.  Try again...
Please enter a number: f
Oops!  That was not a valid number.  Try again...
Please enter a number: f
Oops!  That was not a valid number.  Try again...
Please enter a number: f
Oops!  That was not a valid number.  Try again...
Please enter a number: f
Oops!  That was not a valid number.  Try again...
Please enter a number: f
Oops!  That was not a valid number.  Try again...
Please enter a number: 1
Thanks, 1 is indeed an integer
